In [1]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import squarify  # для treemap
from lifelines import KaplanMeierFitter  # для survival analysis
from scipy import stats
import networkx as nx  # для network analysis

# Настройки для отображения
warnings.filterwarnings('ignore')

# Безопасная настройка стиля matplotlib
try:
    plt.style.use('seaborn-v0_8')
except OSError:
    try:
        plt.style.use('seaborn')
    except OSError:
        plt.style.use('default')
        print("Используется стандартный стиль matplotlib")

sns.set_palette("husl")

# Настройки pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

print("Библиотеки успешно импортированы")


ModuleNotFoundError: No module named 'squarify'

In [ ]:
# Загрузка данных
print("Загружаем данные о мошенничестве...")

# Загрузка основного датасета
df = pd.read_parquet('data/transaction_fraud_data.parquet')

# Загрузка данных по валютным курсам
currency_data = pd.read_parquet('data/historical_currency_exchange.parquet')

# Функция для конвертации валют в USD
def convert_to_usd(df, currency_rates):
    """Конвертация amount в USD с использованием курсов валют"""
    df_converted = df.copy()
    
    # Создаем словарь курсов валют (используем последние доступные курсы)
    latest_rates = currency_rates.iloc[-1]  # Последняя дата
    
    # Маппинг стран к валютам
    country_currency_map = {
        'Nigeria': 'NGN',
        'Brazil': 'BRL', 
        'Russia': 'RUB',
        'Mexico': 'MXN',
        'Singapore': 'SGD',
        'Canada': 'CAD',
        'Australia': 'AUD',
        'United Kingdom': 'GBP',
        'European Union': 'EUR',
        'Japan': 'JPY',
        'USA': 'USD'
    }
    
    # Создаем столбец amount_usd
    df_converted['amount_usd'] = df_converted['amount'].copy()
    
    for country, currency_code in country_currency_map.items():
        if country in df_converted['country'].values and currency_code in latest_rates.index:
            mask = df_converted['country'] == country
            if currency_code != 'USD':
                # Конвертируем в USD
                exchange_rate = latest_rates[currency_code]
                df_converted.loc[mask, 'amount_usd'] = df_converted.loc[mask, 'amount'] / exchange_rate
                print(f"Конвертирован {country}: {currency_code} -> USD (курс: {exchange_rate:.4f})")
            else:
                print(f"{country}: уже в USD")
    
    return df_converted

# Применяем конвертацию валют
df = convert_to_usd(df, currency_data)

print(f"Данные успешно загружены!")
print(f"Размер основного датасета: {df.shape[0]:,} строк, {df.shape[1]} столбцов")
print(f"Размер данных по курсам валют: {currency_data.shape[0]:,} строк, {currency_data.shape[1]} столбцов")

print("\nСтолбцы в основном датасете:")
print(df.columns.tolist())

print("\nПервые 3 строки:")
print(df.head(3))


In [ ]:
# Первичный анализ данных
def analyze_dataset(df):
    """Функция для первичного анализа датасета"""
    print("=== БАЗОВАЯ ИНФОРМАЦИЯ О ДАТАСЕТЕ ===")
    print(f"Размер датасета: {df.shape[0]:,} строк, {df.shape[1]} столбцов")
    print(f"Объем данных в памяти: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print("\n=== ТИПЫ ДАННЫХ ===")
    print(df.dtypes)
    
    print("\n=== ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ ===")
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    missing_info = pd.DataFrame({
        'Пропущенные значения': missing_data,
        'Процент': missing_percent
    }).sort_values('Пропущенные значения', ascending=False)
    print(missing_info[missing_info['Пропущенные значения'] > 0])
    
    print("\n=== ПЕРВЫЕ 5 СТРОК ===")
    print(df.head())
    
    return missing_info

# Запуск анализа данных
missing_info = analyze_dataset(df)

print("Анализ данных завершен.")


In [ ]:
# Анализ информативности столбцов
def analyze_column_informativeness(df):
    """Анализ информативности столбцов для принятия решения об удалении"""
    
    # Проверяем какие столбцы есть в данных
    available_analysis_columns = ['device_fingerprint', 'ip_address', 'transaction_id', 
                                 'customer_id', 'card_number']
    columns_to_analyze = [col for col in available_analysis_columns if col in df.columns]
    
    results = {}
    
    print("=== АНАЛИЗ ИНФОРМАТИВНОСТИ СТОЛБЦОВ ===\n")
    
    for col in columns_to_analyze:
        if col in df.columns:
            unique_count = df[col].nunique()
            total_count = len(df)
            unique_ratio = unique_count / total_count
            
            print(f"СТОЛБЕЦ: {col}")
            print(f"  Уникальных значений: {unique_count:,} из {total_count:,}")
            print(f"  Доля уникальных: {unique_ratio:.4f} ({unique_ratio*100:.2f}%)")
            
            # Анализ информативности
            if unique_ratio > 0.95:
                recommendation = "УДАЛИТЬ - слишком высокая уникальность"
            elif unique_ratio < 0.01:
                recommendation = "УДАЛИТЬ - слишком низкая вариативность"
            else:
                recommendation = "ОСТАВИТЬ - потенциально информативен"
            
            print(f"  Рекомендация: {recommendation}")
            
            # Примеры значений
            print(f"  Примеры значений: {df[col].head(3).tolist()}")
            print()
            
            results[col] = {
                'unique_count': unique_count,
                'unique_ratio': unique_ratio,
                'recommendation': recommendation
            }
        else:
            print(f"Столбец {col} не найден в данных")
    
    return results

# Создание функции для удаления неинформативных столбцов
def remove_uninformative_columns(df, columns_analysis):
    """Удаление неинформативных столбцов на основе анализа"""
    columns_to_remove = []
    
    for col, analysis in columns_analysis.items():
        if "УДАЛИТЬ" in analysis['recommendation']:
            columns_to_remove.append(col)
    
    print(f"Столбцы для удаления: {columns_to_remove}")
    
    if columns_to_remove:
        df_cleaned = df.drop(columns=columns_to_remove)
        print(f"Размер данных до очистки: {df.shape}")
        print(f"Размер данных после очистки: {df_cleaned.shape}")
        return df_cleaned, columns_to_remove
    else:
        print("Столбцы для удаления не найдены")
        return df, []

# Запуск анализа информативности столбцов
columns_analysis = analyze_column_informativeness(df)
df_cleaned, removed_columns = remove_uninformative_columns(df, columns_analysis)

print("Анализ информативности завершен")


In [ ]:
# Создание дополнительных признаков
def create_additional_features(df):
    """Создание дополнительных признаков для анализа"""
    df_enhanced = df.copy()
    
    print("=== СОЗДАНИЕ ДОПОЛНИТЕЛЬНЫХ ПРИЗНАКОВ ===")
    
    # Преобразование timestamp в datetime если нужно
    if 'timestamp' in df_enhanced.columns:
        if df_enhanced['timestamp'].dtype == 'object':
            df_enhanced['timestamp'] = pd.to_datetime(df_enhanced['timestamp'])
        
        # Извлечение временных признаков
        df_enhanced['hour'] = df_enhanced['timestamp'].dt.hour
        df_enhanced['day_of_week'] = df_enhanced['timestamp'].dt.dayofweek
        df_enhanced['day_name'] = df_enhanced['timestamp'].dt.day_name()
        
        print("✓ Созданы временные признаки: hour, day_of_week, day_name")
    
    # Категоризация размера транзакции
    # Проверяем наличие столбца amount_usd (после конвертации валют)
    amount_col = 'amount_usd' if 'amount_usd' in df_enhanced.columns else 'amount'
    
    if amount_col in df_enhanced.columns:
        # Определение квартилей для категоризации
        q25 = df_enhanced[amount_col].quantile(0.25)
        q50 = df_enhanced[amount_col].quantile(0.50)
        q75 = df_enhanced[amount_col].quantile(0.75)
        q95 = df_enhanced[amount_col].quantile(0.95)
        
        def categorize_transaction_size(amount):
            if amount <= q25:
                return 'Малая'
            elif amount <= q50:
                return 'Средняя'
            elif amount <= q75:
                return 'Большая'
            elif amount <= q95:
                return 'Очень большая'
            else:
                return 'Экстремальная'
        
        df_enhanced['transaction_size_category'] = df_enhanced[amount_col].apply(categorize_transaction_size)
        
        print(f"✓ Создан признак transaction_size_category")
        print(f"  Границы: Малая ≤{q25:.2f}, Средняя ≤{q50:.2f}, Большая ≤{q75:.2f}, Очень большая ≤{q95:.2f}")
    
    # Обработка last_hour_activity (в данных это один столбец, возможно JSON/dict)
    if 'last_hour_activity' in df_enhanced.columns:
        print("✓ Найден столбец last_hour_activity")
        # Если это строка JSON, можно попробовать распарсить
        try:
            # Попробуем извлечь информацию из last_hour_activity
            sample_value = df_enhanced['last_hour_activity'].iloc[0]
            print(f"Пример значения last_hour_activity: {sample_value}")
            
            # Простые метрики на основе last_hour_activity
            df_enhanced['has_hour_activity'] = df_enhanced['last_hour_activity'].notna().astype(int)
            print("✓ Создан признак has_hour_activity")
            
        except Exception as e:
            print(f"Не удалось обработать last_hour_activity: {e}")
    
    # Соотношение с текущей транзакцией (используем правильное название столбца)
    if amount_col in df_enhanced.columns:
        # Создаем логарифм суммы в USD для корректного анализа
        df_enhanced['amount_log'] = np.log1p(df_enhanced[amount_col])
        print(f"✓ Создан признак amount_log на основе {amount_col}")
        
        # Добавляем информацию о валютах после конвертации
        if amount_col == 'amount_usd':
            print("✓ amount_log создан на основе USD - все суммы сконвертированы для корректного сравнения")
        else:
            print("⚠️ ВНИМАНИЕ: amount_log создан на основе исходных валют, не USD!")
    
    # Признак времени суток
    if 'hour' in df_enhanced.columns:
        def get_time_period(hour):
            if 6 <= hour < 12:
                return 'Утро'
            elif 12 <= hour < 18:
                return 'День'
            elif 18 <= hour < 22:
                return 'Вечер'
            else:
                return 'Ночь'
        
        df_enhanced['time_period'] = df_enhanced['hour'].apply(get_time_period)
        print("✓ Создан признак time_period")
    
    print(f"\nРазмер данных: {df_enhanced.shape}")
    print(f"Добавлено признаков: {df_enhanced.shape[1] - df.shape[1]}")
    
    return df_enhanced

# Запуск создания дополнительных признаков
df_with_features = create_additional_features(df_cleaned)

print("Создание признаков завершено")


In [ ]:
# Анализ распределения целевой переменной
def analyze_target_variable(df):
    """Анализ распределения целевой переменной is_fraud"""
    
    if 'is_fraud' not in df.columns:
        print("Столбец 'is_fraud' не найден в данных")
        return
    
    print("=== АНАЛИЗ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ ===")
    
    # Основная статистика
    fraud_counts = df['is_fraud'].value_counts()
    fraud_percentages = df['is_fraud'].value_counts(normalize=True) * 100
    
    # Таблица с результатами
    fraud_summary = pd.DataFrame({
        'Количество': fraud_counts,
        'Процент': fraud_percentages
    })
    fraud_summary.index = ['Легитимные', 'Мошеннические']
    
    print("РАСПРЕДЕЛЕНИЕ ТРАНЗАКЦИЙ:")
    print(fraud_summary)
    
    # Создание графиков
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # График 1: Столбчатая диаграмма количества
    fraud_counts.plot(kind='bar', ax=axes[0], color=['skyblue', 'salmon'])
    axes[0].set_title('Распределение транзакций по типу')
    axes[0].set_xlabel('Тип транзакции')
    axes[0].set_ylabel('Количество')
    axes[0].set_xticklabels(['Легитимные', 'Мошеннические'], rotation=0)
    
    # Добавление значений на столбцы
    for i, v in enumerate(fraud_counts.values):
        axes[0].text(i, v + max(fraud_counts) * 0.01, f'{v:,}', ha='center')
    
    # График 2: Круговая диаграмма
    axes[1].pie(fraud_counts.values, labels=['Легитимные', 'Мошеннические'], 
                autopct='%1.2f%%', colors=['skyblue', 'salmon'])
    axes[1].set_title('Доля мошеннических транзакций')
    
    # График 3: Доля мошенничества
    fraud_rate = fraud_percentages[1]
    axes[2].bar(['Fraud Rate'], [fraud_rate], color='salmon', alpha=0.7)
    axes[2].set_ylim(0, 100)
    axes[2].set_title(f'Процент мошенничества: {fraud_rate:.3f}%')
    axes[2].set_ylabel('Процент мошеннических транзакций (меньше лучше)')
    
    # Добавление значения на столбец
    axes[2].text(0, fraud_rate + 2, f'{fraud_rate:.3f}%', ha='center')
    
    plt.tight_layout()
    plt.show()
    
    # Анализ дисбаланса классов
    print(f"\nАНАЛИЗ БАЛАНСА КЛАССОВ:")
    print(f"Соотношение легитимных к мошенническим: {fraud_counts[0] / fraud_counts[1]:.1f}:1")
    
    if fraud_rate < 5:
        print(f"СИЛЬНЫЙ ДИСБАЛАНС КЛАССОВ (fraud rate = {fraud_rate:.3f}%)")
        print("Рекомендуется использовать техники работы с несбалансированными данными")
    elif fraud_rate < 10:
        print(f"УМЕРЕННЫЙ ДИСБАЛАНС КЛАССОВ (fraud rate = {fraud_rate:.3f}%)")
    else:
        print(f"ОТНОСИТЕЛЬНО СБАЛАНСИРОВАННЫЕ КЛАССЫ (fraud rate = {fraud_rate:.3f}%)")
    
    return fraud_summary

# Анализ fraud rate по времени
def analyze_fraud_trends_over_time(df):
    """Анализ трендов мошенничества по времени"""
    
    if 'timestamp' not in df.columns or 'is_fraud' not in df.columns:
        print("Необходимые столбцы 'timestamp' и 'is_fraud' не найдены")
        return
    
    print("\n=== ТРЕНДЫ МОШЕННИЧЕСТВА ПО ВРЕМЕНИ ===")
    
    # Подготовка данных
    df_time = df.copy()
    df_time['date'] = df_time['timestamp'].dt.date
    
    # Группировка по датам
    daily_stats = df_time.groupby('date').agg({
        'is_fraud': ['count', 'sum', 'mean']
    }).reset_index()
    
    daily_stats.columns = ['date', 'total_transactions', 'fraud_count', 'fraud_rate']
    daily_stats['fraud_rate'] = daily_stats['fraud_rate'] * 100
    
    # График тренда
    fig, axes = plt.subplots(2, 1, figsize=(15, 10))
    
    # График 1: Количество транзакций и мошенничества по дням (реальные значения)
    axes[0].plot(daily_stats['date'], daily_stats['total_transactions'], 
                label='Общее количество транзакций', color='blue', alpha=0.7, linewidth=2)
    
    # Создаем вторую ось Y для мошеннических транзакций (реальные значения)
    ax0_twin = axes[0].twinx()
    ax0_twin.plot(daily_stats['date'], daily_stats['fraud_count'], 
                  label='Мошеннические транзакции', color='red', alpha=0.7, linewidth=2)
    
    axes[0].set_ylabel('Общее количество транзакций', color='blue')
    ax0_twin.set_ylabel('Мошеннические транзакции (реальные значения)', color='red')
    axes[0].tick_params(axis='y', labelcolor='blue')
    ax0_twin.tick_params(axis='y', labelcolor='red')
    
    axes[0].set_title('Динамика транзакций по дням (реальные значения)')
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(True, alpha=0.3)
    
    # Добавляем легенды
    axes[0].legend(loc='upper left')
    ax0_twin.legend(loc='upper right')
    
    # График 2: Fraud rate по дням
    axes[1].plot(daily_stats['date'], daily_stats['fraud_rate'], 
                color='orange', marker='o', markersize=3)
    axes[1].set_title('Динамика процента мошенничества по дням')
    axes[1].set_ylabel('Fraud Rate (%)')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Статистика по трендам
    print(f"Средний fraud rate: {daily_stats['fraud_rate'].mean():.3f}%")
    print(f"Минимальный fraud rate: {daily_stats['fraud_rate'].min():.3f}%")
    print(f"Максимальный fraud rate: {daily_stats['fraud_rate'].max():.3f}%")
    print(f"Стандартное отклонение fraud rate: {daily_stats['fraud_rate'].std():.3f}%")
    
    return daily_stats

# Запуск анализа целевой переменной
fraud_summary = analyze_target_variable(df_with_features)
daily_trends = analyze_fraud_trends_over_time(df_with_features)

print("Анализ целевой переменной завершен")


In [ ]:
# Анализ распределения ключевых переменных
def analyze_amount_distribution(df):
    """Анализ распределения сумм транзакций"""
    
    # Определяем правильное название столбца (приоритет USD после конвертации)
    amount_col = 'amount_usd' if 'amount_usd' in df.columns else 'amount'
    
    if amount_col not in df.columns:
        print(f"Столбец '{amount_col}' не найден")
        print(f"Доступные столбцы: {list(df.columns)}")
        return
    
    print(f"=== АНАЛИЗ РАСПРЕДЕЛЕНИЯ СУММ ТРАНЗАКЦИЙ ({amount_col}) ===")
    
    # Основная статистика
    print(f"ОПИСАТЕЛЬНАЯ СТАТИСТИКА {amount_col}:")
    print(df[amount_col].describe())
    
    # Создание графиков (убираем один график)
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # График 1: Распределение с логарифмической шкалой
    df[amount_col].hist(bins=50, alpha=0.7, ax=axes[0])
    axes[0].set_yscale('log')
    axes[0].set_title(f'Распределение сумм (логарифмическая шкала)')
    # Исправляем отображение валюты - все конвертировано в USD
    currency_label = "USD (все валюты сконвертированы)" if amount_col == 'amount_usd' else "исходная валюта"
    axes[0].set_xlabel(f'Сумма транзакции ({currency_label})')
    axes[0].set_ylabel('Частота транзакций (логарифм)')
    
    # График 2: Распределение по типу транзакции (если есть is_fraud)
    if 'is_fraud' in df.columns:
        for fraud_type in [0, 1]:
            data = df[df['is_fraud'] == fraud_type][amount_col]
            label = 'Мошеннические (хуже)' if fraud_type == 1 else 'Легитимные (лучше)'
            data.hist(bins=50, alpha=0.6, ax=axes[1], label=label, density=True)
        
        axes[1].legend()
        axes[1].set_title(f'Сравнение распределения сумм по типу транзакции')
        # Исправляем отображение валюты - все конвертировано в USD
        axes[1].set_xlabel(f'Сумма транзакции ({currency_label})')
        axes[1].set_ylabel('Плотность распределения')
    
    # График 3: Box plot по квартилям
    quartiles = pd.qcut(df[amount_col], q=4, labels=['Q1 (25%)', 'Q2 (50%)', 'Q3 (75%)', 'Q4 (95%)'])
    df_with_quartiles = df.copy()
    df_with_quartiles['quartile'] = quartiles
    sns.boxplot(data=df_with_quartiles, x='quartile', y=amount_col, ax=axes[2])
    axes[2].set_title(f'Квартильное распределение сумм транзакций')
    axes[2].set_xlabel('Квартили (Q1-малые суммы, Q4-большие суммы)')
    axes[2].set_ylabel(f'Сумма транзакции ({amount_col})')
    
    plt.tight_layout()
    plt.show()
    
    # Анализ выбросов
    Q1 = df[amount_col].quantile(0.25)
    Q3 = df[amount_col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[amount_col] < lower_bound) | (df[amount_col] > upper_bound)]
    
    print(f"\nАНАЛИЗ ВЫБРОСОВ:")
    print(f"Выбросов: {len(outliers):,} ({len(outliers)/len(df)*100:.2f}%)")
    print(f"Верхняя граница: {upper_bound:,.2f}")
    print(f"Максимальная сумма: {df[amount_col].max():,.2f}")

def analyze_categorical_distributions(df):
    """Анализ распределения категориальных переменных"""
    
    categorical_columns = ['country', 'vendor_category', 'device', 'channel']
    available_columns = [col for col in categorical_columns if col in df.columns]
    
    if not available_columns:
        print("Категориальные столбцы не найдены")
        return
    
    print("=== АНАЛИЗ КАТЕГОРИАЛЬНЫХ ПЕРЕМЕННЫХ ===")
    
    # Создание графиков
    n_cols = 2
    n_rows = (len(available_columns) + 1) // 2
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 6 * n_rows))
    
    # Правильная обработка axes для разных случаев
    if n_rows == 1 and n_cols == 1:
        axes = [axes]  # Один график
    elif n_rows == 1:
        axes = axes  # Одна строка, несколько столбцов
    else:
        axes = axes.flatten()  # Несколько строк и столбцов
    
    for i, col in enumerate(available_columns):
        # Count plot
        top_values = df[col].value_counts().head(10)  # Топ-10 значений
        
        if len(top_values) > 0:
            top_values.plot(kind='bar', ax=axes[i])
            axes[i].set_title(f'Топ-10 популярных значений: {col}')
            
            # Улучшенные подписи осей
            if col == 'country':
                axes[i].set_xlabel('Страны (по убыванию активности)')
                axes[i].set_ylabel('Количество транзакций (больше = выше активность)')
            elif col == 'vendor_category':
                axes[i].set_xlabel('Категории поставщиков')
                axes[i].set_ylabel('Количество транзакций (больше = популярнее)')
            elif col == 'device':
                axes[i].set_xlabel('Типы устройств')
                axes[i].set_ylabel('Количество использований (больше = популярнее)')
            elif col == 'channel':
                axes[i].set_xlabel('Каналы транзакций')
                axes[i].set_ylabel('Количество транзакций (больше = активнее канал)')
            else:
                axes[i].set_xlabel(f'{col} (категории)')
                axes[i].set_ylabel('Количество записей')
            
            axes[i].tick_params(axis='x', rotation=45)
            
            # Добавление значений на столбцы с процентами (исправляем накладывание)
            total_count = df[col].count()
            for j, v in enumerate(top_values.values):
                percentage = (v / total_count) * 100
                # Процент сверху, абсолютное значение снизу (табуляция)
                axes[i].text(j, v + max(top_values) * 0.05, f'{percentage:.1f}%', 
                           ha='center', fontsize=8, weight='bold')
                axes[i].text(j, v + max(top_values) * 0.02, f'{v:,}', 
                           ha='center', fontsize=7, alpha=0.8)
        
        # Статистика для каждого столбца
        unique_count = df[col].nunique()
        print(f"\n{col.upper()}:")
        print(f"  Уникальных значений: {unique_count:,}")
        print(f"  Топ-5 значений:")
        print(f"  {df[col].value_counts().head().to_dict()}")
    
    # Скрытие пустых подграфиков
    if len(available_columns) < len(axes):
        for j in range(len(available_columns), len(axes)):
            axes[j].set_visible(False)
    
    plt.tight_layout()
    plt.show()

def analyze_activity_metrics(df):
    """Анализ метрик активности и других численных переменных"""
    
    # Ищем численные столбцы для анализа
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Исключаем ID столбцы и целевую переменную
    exclude_cols = ['transaction_id', 'customer_id', 'card_number', 'is_fraud']
    analysis_columns = [col for col in numeric_cols if col not in exclude_cols]
    
    if not analysis_columns:
        print("Численные столбцы для анализа не найдены")
        return
    
    print("=== АНАЛИЗ ЧИСЛЕННЫХ ПРИЗНАКОВ ===")
    print(f"Анализируемые столбцы: {analysis_columns}")
    
    # Убираем ненужные графики: amount (оригинальный) и has_hour_activity
    columns_to_exclude = ['amount', 'has_hour_activity']
    filtered_columns = [col for col in analysis_columns if col not in columns_to_exclude]
    columns_to_plot = filtered_columns[:4]  # Оставляем только 4 графика
    
    if len(columns_to_plot) == 0:
        print("Нет столбцов для построения графиков")
        return
    
    # Создание графиков в 2 столбца
    n_cols = 2
    n_rows = (len(columns_to_plot) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 6 * n_rows))
    
    # Правильная обработка axes
    if n_rows == 1 and n_cols == 1:
        axes = [axes]
    elif n_rows == 1:
        axes = list(axes) if hasattr(axes, '__iter__') else [axes]
    else:
        axes = axes.flatten()
    
    for i, col in enumerate(columns_to_plot):
        try:
            # Проверяем, что в столбце есть численные данные
            if df[col].dtype in ['object', 'string']:
                continue
                
            # Гистограмма
            df[col].hist(bins=30, alpha=0.7, ax=axes[i])
            axes[i].set_title(f'Распределение {col}')
            
            # Улучшенные подписи для численных признаков
            if 'amount' in col.lower():
                axes[i].set_xlabel(f'{col} (сумма)')
                axes[i].set_ylabel('Частота транзакций')
            elif 'activity' in col.lower():
                axes[i].set_xlabel(f'{col} (активность)')
                axes[i].set_ylabel('Частота значений активности')
            elif col in ['is_weekend', 'is_outside_home_country', 'is_high_risk_vendor']:
                axes[i].set_xlabel(f'{col} (0=нет, 1=да)')
                axes[i].set_ylabel('Количество случаев')
            else:
                axes[i].set_xlabel(f'{col} (значения)')
                axes[i].set_ylabel('Частота встречаемости')
            
            # Добавляем статистику на график (как в категориальных переменных)
            mean_val = df[col].mean()
            median_val = df[col].median()
            std_val = df[col].std()
            
            # Добавляем текст со статистикой на график
            stats_text = f'Среднее: {mean_val:.2f}\nМедиана: {median_val:.2f}\nСтд.откл: {std_val:.2f}'
            axes[i].text(0.02, 0.98, stats_text, transform=axes[i].transAxes, 
                        verticalalignment='top', fontsize=8, weight='bold',
                        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
            
            # Статистика в консоль
            print(f"\n{col}:")
            print(f"  Среднее: {mean_val:.4f}")
            print(f"  Медиана: {median_val:.4f}")
            print(f"  Стд. отклонение: {std_val:.4f}")
            print(f"  Мин: {df[col].min():.4f}, Макс: {df[col].max():.4f}")
            
        except Exception as e:
            print(f"Ошибка при анализе столбца {col}: {e}")
            axes[i].text(0.5, 0.5, f'Ошибка: {col}', ha='center', va='center', transform=axes[i].transAxes)
    
    # Скрытие пустых подграфиков
    for j in range(len(columns_to_plot), len(axes)):
        axes[j].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    # Дополнительный анализ last_hour_activity если есть
    if 'last_hour_activity' in df.columns:
        print(f"\nДОПОЛНИТЕЛЬНЫЙ АНАЛИЗ last_hour_activity:")
        sample_values = df['last_hour_activity'].dropna().head(5)
        print(f"Примеры значений: {sample_values.tolist()}")
        print(f"Пропущенных значений: {df['last_hour_activity'].isna().sum()}")
        print(f"Тип данных: {df['last_hour_activity'].dtype}")

# Запуск анализа распределений
analyze_amount_distribution(df_with_features)
analyze_categorical_distributions(df_with_features)
analyze_activity_metrics(df_with_features)

print("Анализ распределений завершен")


In [ ]:
# Анализ паттернов мошенничества
def analyze_fraud_by_amount(df):
    """Анализ fraud rate по диапазонам сумм"""
    
    # Определяем правильное название столбца (приоритет USD после конвертации)
    amount_col = 'amount_usd' if 'amount_usd' in df.columns else 'amount'
    
    if amount_col not in df.columns or 'is_fraud' not in df.columns:
        print(f"Необходимые столбцы не найдены. Нужны: {amount_col}, is_fraud")
        print(f"Доступные столбцы: {list(df.columns)}")
        return
    
    print(f"=== АНАЛИЗ МОШЕННИЧЕСТВА ПО СУММАМ ТРАНЗАКЦИЙ ({amount_col}) ===")
    
    # Создание диапазонов сумм на основе квантилей для более равномерного распределения
    max_amount = df[amount_col].max()
    amount_bins = [0, 50, 100, 200, 500, 1000, 2000, 5000, max_amount]
    amount_labels = ['0-50', '50-100', '100-200', '200-500', '500-1k', '1k-2k', '2k-5k', f'5k-{int(max_amount)}']
    
    df_analysis = df.copy()
    df_analysis['amount_range'] = pd.cut(df_analysis[amount_col], 
                                        bins=amount_bins, 
                                        labels=amount_labels, 
                                        include_lowest=True)
    
    # Группировка и расчет fraud rate
    fraud_by_amount = df_analysis.groupby('amount_range').agg({
        'is_fraud': ['count', 'sum', 'mean']
    }).reset_index()
    
    fraud_by_amount.columns = ['amount_range', 'total_transactions', 'fraud_count', 'fraud_rate']
    fraud_by_amount['fraud_rate'] = fraud_by_amount['fraud_rate'] * 100
    
    # Создание графика (оставляем только один график)
    fig, ax = plt.subplots(1, 1, figsize=(14, 8))
    
    # График: Количество транзакций и мошенничества
    x = range(len(fraud_by_amount))
    width = 0.35
    
    ax.bar([i - width/2 for i in x], fraud_by_amount['total_transactions'], 
           width, label='Всего транзакций (больше = активнее)', alpha=0.7)
    ax.bar([i + width/2 for i in x], fraud_by_amount['fraud_count'], 
           width, label='Мошеннические (меньше лучше)', alpha=0.7, color='red')
    
    ax.set_title('Объем транзакций vs мошенничество по суммам (в USD)')
    ax.set_xlabel('Диапазоны сумм транзакций (в USD)')
    ax.set_ylabel('Количество транзакций')
    ax.set_xticks(x)
    ax.set_xticklabels(fraud_by_amount['amount_range'], rotation=45)
    ax.legend()
    
    # Добавляем fraud rate как текст с табуляцией (как в категориальных переменных)
    for i, (total, fraud, rate) in enumerate(zip(fraud_by_amount['total_transactions'], 
                                                 fraud_by_amount['fraud_count'], 
                                                 fraud_by_amount['fraud_rate'])):
        max_height = max(total, fraud)
        ax.text(i, max_height + max_height * 0.08, f'{rate:.1f}%', 
                ha='center', fontsize=10, weight='bold')
        

    
    plt.tight_layout()
    plt.show()
    
    # Таблица с результатами
    print("\nТАБЛИЦА FRAUD RATE ПО ДИАПАЗОНАМ СУММ:")
    print(fraud_by_amount.to_string(index=False))
    
    return fraud_by_amount

def analyze_fraud_by_category(df, category_column, top_n=None):
    """Анализ fraud rate по категориям"""
    
    if category_column not in df.columns or 'is_fraud' not in df.columns:
        print(f"Столбец '{category_column}' или 'is_fraud' не найден")
        return
    
    print(f"=== АНАЛИЗ МОШЕННИЧЕСТВА ПО {category_column.upper()} ===")
    
    # Группировка и расчет fraud rate
    fraud_by_category = df.groupby(category_column).agg({
        'is_fraud': ['count', 'sum', 'mean']
    }).reset_index()
    
    fraud_by_category.columns = [category_column, 'total_transactions', 'fraud_count', 'fraud_rate']
    fraud_by_category['fraud_rate'] = fraud_by_category['fraud_rate'] * 100
    
    # Фильтрация только категорий с достаточным количеством транзакций
    min_transactions = 100  # Минимум для статистической значимости
    fraud_by_category = fraud_by_category[fraud_by_category['total_transactions'] >= min_transactions]
    
    # Показываем все категории, отсортированные по fraud rate
    if top_n is None:
        top_n = len(fraud_by_category)  # Показать все
    
    top_fraud_categories = fraud_by_category.nlargest(top_n, 'fraud_rate')
    
    # График
    plt.figure(figsize=(14, max(8, len(top_fraud_categories) * 0.5)))
    plt.barh(range(len(top_fraud_categories)), top_fraud_categories['fraud_rate'], 
             color='salmon', alpha=0.7)
    plt.yticks(range(len(top_fraud_categories)), top_fraud_categories[category_column])
    plt.xlabel('Процент мошенничества (меньше лучше)')
    
    if len(top_fraud_categories) == len(fraud_by_category):
        plt.title(f'Все категории по проценту мошенничества: {category_column}')
    else:
        plt.title(f'Топ-{top_n} категорий по проценту мошенничества: {category_column}')
    
    plt.gca().invert_yaxis()
    
    # Добавление значений с табуляцией (как в категориальных переменных)
    for i, (fraud_rate, transactions) in enumerate(zip(top_fraud_categories['fraud_rate'], 
                                                       top_fraud_categories['total_transactions'])):
        # Процент сверху, абсолютное значение снизу (табуляция)
        plt.text(fraud_rate, i + 0.2, f'{fraud_rate:.1f}%', 
                va='center', fontsize=9, weight='bold', ha='left')
        plt.text(fraud_rate, i - 0.2, f'({transactions:,})', 
                va='center', fontsize=8, alpha=0.8, ha='left')
    
    plt.tight_layout()
    plt.show()
    
    # Таблица с результатами
    if len(top_fraud_categories) == len(fraud_by_category):
        print(f"\nВСЕ КАТЕГОРИИ ПО FRAUD RATE:")
    else:
        print(f"\nТОП-{top_n} КАТЕГОРИЙ ПО FRAUD RATE:")
    print(top_fraud_categories.to_string(index=False))
    
    print(f"\nСТАТИСТИКА:")
    print(f"Средний fraud rate: {fraud_by_category['fraud_rate'].mean():.3f}%")
    print(f"Медианный fraud rate: {fraud_by_category['fraud_rate'].median():.3f}%")
    print(f"Максимальный fraud rate: {fraud_by_category['fraud_rate'].max():.3f}%")
    
    return fraud_by_category

def analyze_fraud_correlations(df):
    """Корреляционный анализ между fraud и числовыми переменными"""
    
    if 'is_fraud' not in df.columns:
        print("Столбец 'is_fraud' не найден")
        return
    
    print("=== КОРРЕЛЯЦИОННЫЙ АНАЛИЗ С МОШЕННИЧЕСТВОМ ===")
    
    # Отбор числовых столбцов
    numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
    
    if 'is_fraud' not in numeric_columns:
        print("is_fraud не является числовым столбцом")
        return
    
    # Расчет корреляций с is_fraud
    correlations = df[numeric_columns].corr()['is_fraud'].sort_values(key=abs, ascending=False)
    
    # Удаление автокорреляции is_fraud с самим собой
    correlations = correlations.drop('is_fraud')
    
    # Топ-15 корреляций
    top_correlations = correlations.head(15)
    
    # График корреляций
    plt.figure(figsize=(12, 8))
    colors = ['red' if x < 0 else 'green' for x in top_correlations.values]
    plt.barh(range(len(top_correlations)), top_correlations.values, color=colors, alpha=0.7)
    plt.yticks(range(len(top_correlations)), top_correlations.index)
    plt.xlabel('Корреляция с is_fraud')
    plt.title('Топ-15 корреляций признаков с мошенничеством')
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.gca().invert_yaxis()
    
    # Добавление значений
    for i, v in enumerate(top_correlations.values):
        plt.text(v + (0.01 if v >= 0 else -0.01), i, f'{v:.3f}', 
                va='center', ha='left' if v >= 0 else 'right')
    
    plt.tight_layout()
    plt.show()
    
    # Heatmap топ коррелирующих признаков
    top_corr_columns = ['is_fraud'] + top_correlations.head(10).index.tolist()
    correlation_matrix = df[top_corr_columns].corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='RdBu_r', center=0, 
                square=True, fmt='.3f')
    plt.title('Heatmap корреляций (топ-10 признаков + is_fraud)')
    plt.tight_layout()
    plt.show()
    
    # Таблица результатов
    print("ТОП-15 КОРРЕЛЯЦИЙ С МОШЕННИЧЕСТВОМ:")
    correlations_df = pd.DataFrame({
        'Признак': top_correlations.index,
        'Корреляция': top_correlations.values
    })
    print(correlations_df.to_string(index=False))
    
    return correlations

# Функция для комплексного анализа паттернов мошенничества
def comprehensive_fraud_analysis(df):
    """Комплексный анализ паттернов мошенничества"""
    
    print("=== КОМПЛЕКСНЫЙ АНАЛИЗ ПАТТЕРНОВ МОШЕННИЧЕСТВА ===")
    
    # Анализ по суммам
    amount_analysis = analyze_fraud_by_amount(df)
    
    # Анализ по категориям
    categories_to_analyze = ['vendor_category', 'country', 'channel', 'device']
    
    for category in categories_to_analyze:
        if category in df.columns:
            print(f"\n{'='*50}")
            category_analysis = analyze_fraud_by_category(df, category)
    
    # Корреляционный анализ
    print(f"\n{'='*50}")
    correlations = analyze_fraud_correlations(df)
    
    return {
        'amount_analysis': amount_analysis,
        'correlations': correlations
    }

# Запуск комплексного анализа паттернов мошенничества
fraud_patterns = comprehensive_fraud_analysis(df_with_features)

print("Анализ паттернов мошенничества завершен")


In [ ]:
# ✅ ЗАПУСК ИСПРАВЛЕННОГО АНАЛИЗА
print("🚀 Все исправления внедрены в notebook!")
print("\n📋 Для запуска анализа выполните ячейки в следующем порядке:")
print("1. Ячейка 1: Импорт библиотек")
print("2. Ячейка 2: Загрузка данных и конвертация валют")
print("3. Ячейки 4-8: Препроцессинг и создание признаков") 
print("4. Ячейки 10-16: Анализ данных с исправленными графиками")
print("\n✨ Основные улучшения:")
print("• Единый масштаб графиков")
print("• Корректные валютные подписи (USD)")
print("• Улучшенное отображение подписей")
print("• Оптимизированное количество графиков")
print("• Проверка USD-конвертации везде")

print("\n🎯 Теперь анализ готов к использованию!")
print("📊 Все графики будут корректно отображать данные в USD с правильными масштабами.")


In [ ]:
# Временной и географический анализ
def analyze_server_load_patterns(df):
    """Анализ паттернов нагрузки для оптимизации серверов"""
    
    df_analysis = df.copy()
    
    # Проверяем и создаем столбец hour если его нет
    if 'hour' not in df_analysis.columns and 'timestamp' in df_analysis.columns:
        print("Создаем столбец 'hour' из timestamp...")
        df_analysis['timestamp'] = pd.to_datetime(df_analysis['timestamp'])
        df_analysis['hour'] = df_analysis['timestamp'].dt.hour
    
    required_columns = ['timestamp', 'country', 'hour']
    missing_columns = [col for col in required_columns if col not in df_analysis.columns]
    
    if missing_columns:
        print(f"Отсутствуют столбцы: {missing_columns}")
        print(f"Доступные столбцы: {list(df_analysis.columns)}")
        return
    
    print("=== АНАЛИЗ ПАТТЕРНОВ НАГРУЗКИ СЕРВЕРОВ ===")
    
    # Используем подготовленные данные
    df_load = df_analysis
    
    # Анализ по часам и странам
    hourly_country_load = df_load.groupby(['hour', 'country']).size().reset_index(name='transaction_count')
    
    # Все страны, отсортированные по общему объему транзакций
    all_countries = df_load['country'].value_counts().index.tolist()
    
    # Используем все страны для анализа
    hourly_country_all = hourly_country_load
    
    # Создание pivot table для heatmap
    heatmap_data = hourly_country_all.pivot(index='country', columns='hour', values='transaction_count')
    heatmap_data = heatmap_data.fillna(0)
    
    # Сортируем страны по общей активности
    country_totals = heatmap_data.sum(axis=1).sort_values(ascending=False)
    heatmap_data = heatmap_data.loc[country_totals.index]
    
    # График 1: Heatmap нагрузки по часам и странам
    plt.figure(figsize=(16, max(8, len(all_countries) * 0.8)))
    sns.heatmap(heatmap_data, annot=False, cmap='YlOrRd', cbar_kws={'label': 'Количество транзакций'})
    plt.title('Heatmap активности: Все страны × Часы дня')
    plt.xlabel('Час дня (0-23)')
    plt.ylabel('Страны (упорядочены по общей активности)')
    plt.tight_layout()
    plt.show()
    
    # Дополнительные heatmap-ы по категориям
    print("\n=== ДОПОЛНИТЕЛЬНЫЕ HEATMAP-Ы ПО КАТЕГОРИЯМ ===")
    
    # Heatmap по vendor_category для всех стран
    if 'vendor_category' in df_load.columns:
        vendor_country_data = df_load.groupby(['country', 'vendor_category']).size().reset_index(name='transaction_count')
        vendor_pivot = vendor_country_data.pivot(index='country', columns='vendor_category', values='transaction_count')
        vendor_pivot = vendor_pivot.fillna(0)
        
        # Сортируем страны по общей активности
        vendor_country_totals = vendor_pivot.sum(axis=1).sort_values(ascending=False)
        vendor_pivot = vendor_pivot.loc[vendor_country_totals.index]
        
        plt.figure(figsize=(14, max(8, len(all_countries) * 0.6)))
        sns.heatmap(vendor_pivot, annot=True, fmt='.0f', cmap='Blues', 
                   cbar_kws={'label': 'Количество транзакций'})
        plt.title('Heatmap: Все страны × Все категории поставщиков')
        plt.xlabel('Категории поставщиков (больше транзакций = популярнее)')
        plt.ylabel('Страны (упорядочены по активности)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    
    # Heatmap по channel для всех стран
    if 'channel' in df_load.columns:
        channel_country_data = df_load.groupby(['country', 'channel']).size().reset_index(name='transaction_count')
        channel_pivot = channel_country_data.pivot(index='country', columns='channel', values='transaction_count')
        channel_pivot = channel_pivot.fillna(0)
        
        # Сортируем страны по общей активности
        channel_country_totals = channel_pivot.sum(axis=1).sort_values(ascending=False)
        channel_pivot = channel_pivot.loc[channel_country_totals.index]
        
        plt.figure(figsize=(12, max(6, len(all_countries) * 0.5)))
        sns.heatmap(channel_pivot, annot=True, fmt='.0f', cmap='Greens',
                   cbar_kws={'label': 'Количество транзакций'})
        plt.title('Heatmap: Все страны × Каналы транзакций')
        plt.xlabel('Каналы транзакций (больше = популярнее канал)')
        plt.ylabel('Страны (упорядочены по активности)')
        plt.tight_layout()
        plt.show()
    
    # Heatmap по device для всех стран
    if 'device' in df_load.columns:
        device_country_data = df_load.groupby(['country', 'device']).size().reset_index(name='transaction_count')
        device_pivot = device_country_data.pivot(index='country', columns='device', values='transaction_count')
        device_pivot = device_pivot.fillna(0)
        
        # Сортируем страны по общей активности
        device_country_totals = device_pivot.sum(axis=1).sort_values(ascending=False)
        device_pivot = device_pivot.loc[device_country_totals.index]
        
        plt.figure(figsize=(15, max(8, len(all_countries) * 0.6)))
        sns.heatmap(device_pivot, annot=True, fmt='.0f', cmap='Purples',
                   cbar_kws={'label': 'Количество транзакций'})
        plt.title('Heatmap: Все страны × Типы устройств')
        plt.xlabel('Типы устройств (больше = популярнее)')
        plt.ylabel('Страны (упорядочены по активности)')
        plt.tight_layout()
        plt.show()
    
    # График 2: Общая нагрузка по часам
    hourly_load = df_load.groupby('hour').size()
    
    plt.figure(figsize=(14, 6))
    plt.plot(hourly_load.index, hourly_load.values, marker='o', linewidth=2, markersize=6)
    plt.title('Нагрузка серверов: количество транзакций по часам')
    plt.xlabel('Час дня (0=полночь, 12=полдень)')
    plt.ylabel('Количество транзакций (больше = выше нагрузка)')
    plt.grid(True, alpha=0.3)
    plt.xticks(range(0, 24))
    
    # Выделение пиковых часов
    peak_hours = hourly_load.nlargest(5)
    for hour, count in peak_hours.items():
        plt.annotate(f'Пик: {count:,}', xy=(hour, count), xytext=(10, 10), 
                    textcoords='offset points', ha='left',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
    
    plt.tight_layout()
    plt.show()
    
    # График 3: Нагрузка по дням недели
    if 'day_of_week' in df_load.columns:
        weekly_load = df_load.groupby('day_of_week').size()
        day_names = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье']
        
        plt.figure(figsize=(12, 6))
        plt.bar(range(7), weekly_load.values, color='lightblue', alpha=0.7)
        plt.title('Серверная нагрузка по дням недели')
        plt.xlabel('Дни недели (важно для планирования выходных)')
        plt.ylabel('Общее количество транзакций (больше = выше нагрузка)')
        plt.xticks(range(7), day_names, rotation=45)
        
        # Добавление значений на столбцы
        for i, v in enumerate(weekly_load.values):
            plt.text(i, v + max(weekly_load) * 0.01, f'{v:,}', ha='center')
        
        plt.tight_layout()
        plt.show()
    
    # Создание таблицы для планирования серверных мощностей
    server_planning_table = create_server_planning_table(df_load, all_countries)
    
    return {
        'hourly_load': hourly_load,
        'heatmap_data': heatmap_data,
        'server_planning': server_planning_table
    }

def create_server_planning_table(df, top_countries):
    """Создание таблицы для планирования серверных мощностей"""
    
    print("\n=== ТАБЛИЦА ДЛЯ ПЛАНИРОВАНИЯ СЕРВЕРНЫХ МОЩНОСТЕЙ ===")
    
    # Общая статистика по часам
    hourly_stats = df.groupby('hour').size().reset_index(name='total_transactions').set_index('hour')
    
    # Статистика по странам и часам
    country_hour_stats = df[df['country'].isin(top_countries)].groupby(['country', 'hour']).size().reset_index(name='transactions')
    
    # Пиковые часы для каждой страны
    country_peaks = country_hour_stats.loc[country_hour_stats.groupby('country')['transactions'].idxmax()]
    
    # Создание сводной таблицы
    planning_table = pd.DataFrame()
    planning_table['hour'] = range(24)
    
    # Безопасное заполнение данных
    load_values = []
    for hour in range(24):
        if hour in hourly_stats.index:
            load_values.append(hourly_stats.loc[hour, 'total_transactions'])
        else:
            load_values.append(0)
    planning_table['total_load'] = load_values
    
    # Определение уровня нагрузки
    def categorize_load(load, percentiles):
        if load >= percentiles[0.95]:
            return 'Критическая'
        elif load >= percentiles[0.80]:
            return 'Высокая'
        elif load >= percentiles[0.50]:
            return 'Средняя'
        else:
            return 'Низкая'
    
    load_percentiles = planning_table['total_load'].quantile([0.5, 0.8, 0.95])
    planning_table['load_category'] = planning_table['total_load'].apply(
        lambda x: categorize_load(x, load_percentiles)
    )
    
    # Рекомендации по масштабированию
    def get_scaling_recommendation(category):
        recommendations = {
            'Критическая': 'Максимальные мощности + резерв',
            'Высокая': 'Увеличенные мощности',
            'Средняя': 'Стандартные мощности',
            'Низкая': 'Минимальные мощности'
        }
        return recommendations.get(category, 'Стандартные мощности')
    
    planning_table['scaling_recommendation'] = planning_table['load_category'].apply(get_scaling_recommendation)
    
    print("РЕКОМЕНДАЦИИ ПО ЧАСАМ:")
    print(planning_table.to_string(index=False))
    
    # Топ-5 пиковых часов
    peak_hours = planning_table.nlargest(5, 'total_load')[['hour', 'total_load', 'load_category']]
    print(f"\nТОП-5 ПИКОВЫХ ЧАСОВ:")
    print(peak_hours.to_string(index=False))
    
    # Рекомендации по странам
    print(f"\nПИКОВЫЕ ЧАСЫ ПО ТОП-СТРАНАМ:")
    for _, row in country_peaks.iterrows():
        print(f"  {row['country']}: час {row['hour']} ({row['transactions']:,} транзакций)")
    
    return planning_table

def analyze_fraud_time_patterns(df):
    """Анализ временных паттернов мошенничества"""
    
    df_analysis = df.copy()
    
    # Создаем столбец hour если его нет
    if 'hour' not in df_analysis.columns and 'timestamp' in df_analysis.columns:
        print("Создаем столбец 'hour' из timestamp...")
        df_analysis['timestamp'] = pd.to_datetime(df_analysis['timestamp'])
        df_analysis['hour'] = df_analysis['timestamp'].dt.hour
    
    required_columns = ['hour', 'is_fraud']
    missing_columns = [col for col in required_columns if col not in df_analysis.columns]
    
    if missing_columns:
        print(f"Отсутствуют столбцы: {missing_columns}")
        print(f"Доступные столбцы: {list(df_analysis.columns)}")
        return
    
    print("\n=== ВРЕМЕННЫЕ ПАТТЕРНЫ МОШЕННИЧЕСТВА ===")
    
    # Fraud rate по часам
    hourly_fraud = df_analysis.groupby('hour').agg({
        'is_fraud': ['count', 'sum', 'mean']
    }).reset_index()
    hourly_fraud.columns = ['hour', 'total_transactions', 'fraud_count', 'fraud_rate']
    hourly_fraud['fraud_rate'] = hourly_fraud['fraud_rate'] * 100
    
    # График fraud rate по часам
    fig, axes = plt.subplots(2, 1, figsize=(15, 12))
    
    # График 1: Fraud rate по часам
    axes[0].plot(hourly_fraud['hour'], hourly_fraud['fraud_rate'], 
                marker='o', linewidth=2, markersize=6, color='red')
    axes[0].set_title('Процент мошенничества по часам дня')
    axes[0].set_xlabel('Час дня (0=полночь, 12=полдень)')
    axes[0].set_ylabel('Процент мошенничества (меньше лучше)')
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xticks(range(0, 24))
    
    # Выделение часов с высоким fraud rate
    high_fraud_hours = hourly_fraud.nlargest(3, 'fraud_rate')
    for _, row in high_fraud_hours.iterrows():
        axes[0].annotate(f'{row["fraud_rate"]:.2f}%', 
                        xy=(row['hour'], row['fraud_rate']), 
                        xytext=(10, 10), textcoords='offset points',
                        bbox=dict(boxstyle='round,pad=0.3', facecolor='red', alpha=0.7))
    
    # График 2: Количество мошеннических транзакций по часам
    axes[1].bar(hourly_fraud['hour'], hourly_fraud['fraud_count'], 
               color='salmon', alpha=0.7)
    axes[1].set_title('Абсолютное количество мошеннических случаев по часам')
    axes[1].set_xlabel('Час дня (0=полночь, 12=полдень)')
    axes[1].set_ylabel('Количество мошеннических транзакций (меньше лучше)')
    axes[1].set_xticks(range(0, 24))
    
    plt.tight_layout()
    plt.show()
    
    # Fraud rate по дням недели (если есть данные)
    if 'day_of_week' in df_analysis.columns:
        daily_fraud = df_analysis.groupby('day_of_week').agg({
            'is_fraud': ['count', 'sum', 'mean']
        }).reset_index()
    elif 'timestamp' in df_analysis.columns:
        # Создаем day_of_week из timestamp
        df_analysis['day_of_week'] = df_analysis['timestamp'].dt.dayofweek
        daily_fraud = df_analysis.groupby('day_of_week').agg({
            'is_fraud': ['count', 'sum', 'mean']
        }).reset_index()
        daily_fraud.columns = ['day_of_week', 'total_transactions', 'fraud_count', 'fraud_rate']
        daily_fraud['fraud_rate'] = daily_fraud['fraud_rate'] * 100
        
        day_names = ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']
        
        plt.figure(figsize=(12, 6))
        plt.bar(range(7), daily_fraud['fraud_rate'], color='orange', alpha=0.7)
        plt.title('Процент мошенничества по дням недели')
        plt.xlabel('Дни недели (Пн-рабочий, Сб-Вс-выходные)')
        plt.ylabel('Процент мошенничества (меньше лучше)')
        plt.xticks(range(7), day_names)
        
        # Добавление значений
        for i, v in enumerate(daily_fraud['fraud_rate']):
            plt.text(i, v + 0.05, f'{v:.2f}%', ha='center')
        
        plt.tight_layout()
        plt.show()
    
    # Таблица с результатами
    print("FRAUD RATE ПО ЧАСАМ:")
    print(hourly_fraud.to_string(index=False))
    
    # Выводы
    peak_fraud_hour = hourly_fraud.loc[hourly_fraud['fraud_rate'].idxmax()]
    low_fraud_hour = hourly_fraud.loc[hourly_fraud['fraud_rate'].idxmin()]
    
    print(f"\nВЫВОДЫ:")
    print(f"Час с максимальным fraud rate: {peak_fraud_hour['hour']} ({peak_fraud_hour['fraud_rate']:.3f}%)")
    print(f"Час с минимальным fraud rate: {low_fraud_hour['hour']} ({low_fraud_hour['fraud_rate']:.3f}%)")
    print(f"Разница между максимумом и минимумом: {peak_fraud_hour['fraud_rate'] - low_fraud_hour['fraud_rate']:.3f}%")
    
    return hourly_fraud

server_analysis = analyze_server_load_patterns(df_with_features)
fraud_time_analysis = analyze_fraud_time_patterns(df_with_features)

print("Функции временного и географического анализа готовы")


In [ ]:
# Запуск альтернативных анализов вместо простых демографических графиков
print("=== ЗАПУСК ПРОДВИНУТЫХ АНАЛИЗОВ ВМЕСТО ПРОСТЫХ ГРАФИКОВ ===")
print("Заменяем малоинформативные графики на:")
print("1. Heatmap комбинированных рисков")
print("2. Анализ скорости мошенничества (velocity)")  
print("3. Кластерный анализ поведения")
print("4. Анализ последовательных паттернов")
print("5. Распределение аномальности")

# Функция будет определена в следующей ячейке
# Здесь мы подготавливаем данные
print(f"\nИспользуем данные: {df_with_features.shape[0]:,} строк, {df_with_features.shape[1]} столбцов")
print("Готовы к запуску альтернативных анализов...")


In [ ]:
# Анализ клиентского поведения
def analyze_demographic_patterns(df):
    """Анализ демографических паттернов мошенничества"""
    
    print("=== ДЕМОГРАФИЧЕСКИЙ АНАЛИЗ КЛИЕНТСКОГО ПОВЕДЕНИЯ ===")
    
    # Проверяем наличие данных о возрасте
    has_age = 'age' in df.columns
    print(f"Данные о возрасте: {'Есть' if has_age else 'Отсутствуют'}")
    
    # Анализ по city_size с коэффициентом риска
    if 'city_size' in df.columns and 'is_fraud' in df.columns:
        print("\n--- АНАЛИЗ РИСКА ПО РАЗМЕРУ ГОРОДА ---")
        
        city_fraud = df.groupby('city_size').agg({
            'is_fraud': ['count', 'sum', 'mean']
        }).reset_index()
        city_fraud.columns = ['city_size', 'total_transactions', 'fraud_count', 'fraud_rate']
        
        # Рассчитываем коэффициент риска относительно общего уровня
        overall_fraud_rate = df['is_fraud'].mean()
        city_fraud['risk_ratio'] = city_fraud['fraud_rate'] / overall_fraud_rate
        city_fraud['fraud_rate'] = city_fraud['fraud_rate'] * 100
        city_fraud['risk_level'] = city_fraud['risk_ratio'].apply(
            lambda x: 'Высокий риск' if x > 1.1 else ('Низкий риск' if x < 0.9 else 'Средний риск')
        )
        

        
        print("АНАЛИЗ РИСКА ПО РАЗМЕРУ ГОРОДА:")
        print(city_fraud[['city_size', 'total_transactions', 'fraud_rate', 'risk_ratio', 'risk_level']].to_string(index=False))
    
    # Анализ is_outside_home_country
    if 'is_outside_home_country' in df.columns and 'is_fraud' in df.columns:
        print("\n--- АНАЛИЗ КЛИЕНТОВ ВНЕ РОДНОЙ СТРАНЫ ---")
        
        outside_fraud = df.groupby('is_outside_home_country').agg({
            'is_fraud': ['count', 'sum', 'mean']
        }).reset_index()
        outside_fraud.columns = ['is_outside_home_country', 'total_transactions', 'fraud_count', 'fraud_rate']
        outside_fraud['fraud_rate'] = outside_fraud['fraud_rate'] * 100
        outside_fraud['location_type'] = outside_fraud['is_outside_home_country'].map({
            False: 'В родной стране', True: 'Вне родной страны'
        })
        

        
        print("FRAUD RATE ПО МЕСТОПОЛОЖЕНИЮ:")
        print(outside_fraud[['location_type', 'total_transactions', 'fraud_count', 'fraud_rate']].to_string(index=False))
    
    # Анализ по типу карты
    if 'card_type' in df.columns and 'is_fraud' in df.columns:
        print("\n--- АНАЛИЗ ПОВЕДЕНЧЕСКИХ ПАТТЕРНОВ ПО ТИПУ КАРТЫ ---")
        
        card_fraud = df.groupby('card_type').agg({
            'is_fraud': ['count', 'sum', 'mean']
        }).reset_index()
        card_fraud.columns = ['card_type', 'total_transactions', 'fraud_count', 'fraud_rate']
        card_fraud['fraud_rate'] = card_fraud['fraud_rate'] * 100
        card_fraud = card_fraud.sort_values('fraud_rate', ascending=False)
        
        # График
        plt.figure(figsize=(12, 6))
        plt.barh(range(len(card_fraud)), card_fraud['fraud_rate'], 
                color='purple', alpha=0.7)
        plt.yticks(range(len(card_fraud)), card_fraud['card_type'])
        plt.xlabel('Процент мошенничества (%)')
        plt.title('Fraud rate по типам карт')
        plt.gca().invert_yaxis()
        
        # Добавляем значения
        for i, (rate, transactions) in enumerate(zip(card_fraud['fraud_rate'], 
                                                   card_fraud['total_transactions'])):
            plt.text(rate + 0.2, i, f'{rate:.1f}% ({transactions:,})', 
                    va='center', fontsize=9)
        
        plt.tight_layout()
        plt.show()
        
        print("FRAUD RATE ПО ТИПАМ КАРТ:")
        print(card_fraud.to_string(index=False))
    
    return {
        'city_analysis': city_fraud if 'city_size' in df.columns else None,
        'location_analysis': outside_fraud if 'is_outside_home_country' in df.columns else None,
        'card_analysis': card_fraud if 'card_type' in df.columns else None
    }

def parse_last_hour_activity(df):
    """Парсинг и анализ данных last_hour_activity"""
    
    print("\n=== АНАЛИЗ АКТИВНОСТИ ЗА ПОСЛЕДНИЙ ЧАС ===")
    
    if 'last_hour_activity' not in df.columns:
        print("Столбец 'last_hour_activity' не найден")
        return None
    
    # Проверяем тип данных
    sample_value = df['last_hour_activity'].iloc[0]
    print(f"Тип данных last_hour_activity: {type(sample_value)}")
    print(f"Пример значения: {sample_value}")
    
    # Парсинг JSON данных
    import ast
    import json
    
    def safe_parse_activity(activity_str):
        """Безопасный парсинг activity данных"""
        if pd.isna(activity_str):
            return None
        try:
            if isinstance(activity_str, str):
                # Попробуем разные методы парсинга
                try:
                    return json.loads(activity_str)
                except:
                    return ast.literal_eval(activity_str)
            elif isinstance(activity_str, dict):
                return activity_str
            else:
                return None
        except:
            return None
    
    # Парсим данные
    print("Парсинг данных last_hour_activity...")
    df_activity = df.copy()
    df_activity['parsed_activity'] = df_activity['last_hour_activity'].apply(safe_parse_activity)
    
    # Фильтруем строки с валидными данными
    valid_activity = df_activity[df_activity['parsed_activity'].notna()].copy()
    print(f"Строк с валидными данными активности: {len(valid_activity):,}")
    
    if len(valid_activity) == 0:
        print("Нет валидных данных для анализа")
        return None
    
    # Извлекаем метрики
    activity_metrics = []
    for _, row in valid_activity.iterrows():
        activity = row['parsed_activity']
        if isinstance(activity, dict):
            metrics = {
                'is_fraud': row['is_fraud'],
                'num_transactions': activity.get('num_transactions', 0),
                'total_amount': activity.get('total_amount', 0),
                'unique_merchants': activity.get('unique_merchants', 0),
                'unique_countries': activity.get('unique_countries', 0),
                'max_single_amount': activity.get('max_single_amount', 0)
            }
            activity_metrics.append(metrics)
    
    if not activity_metrics:
        print("Не удалось извлечь метрики")
        return None
    
    df_metrics = pd.DataFrame(activity_metrics)
    
    # Создаем дополнительные метрики
    df_metrics['avg_transaction_amount'] = df_metrics['total_amount'] / (df_metrics['num_transactions'] + 0.001)
    df_metrics['max_to_total_ratio'] = df_metrics['max_single_amount'] / (df_metrics['total_amount'] + 0.001)
    df_metrics['transactions_per_merchant'] = df_metrics['num_transactions'] / (df_metrics['unique_merchants'] + 0.001)
    
    print(f"Успешно обработано {len(df_metrics):,} записей")
    
    return df_metrics

def analyze_last_hour_activity_patterns(df_metrics):
    """Анализ паттернов активности за последний час"""
    
    if df_metrics is None or len(df_metrics) == 0:
        print("Нет данных для анализа паттернов активности")
        return
    
    print("\n--- СРАВНИТЕЛЬНЫЙ АНАЛИЗ АКТИВНОСТИ МОШЕННИКОВ VS ЛЕГИТИМНЫХ КЛИЕНТОВ ---")
    
    # Разделяем на мошенников и легитимных
    fraud_metrics = df_metrics[df_metrics['is_fraud'] == True]
    legit_metrics = df_metrics[df_metrics['is_fraud'] == False]
    
    print(f"Мошеннические транзакции: {len(fraud_metrics):,}")
    print(f"Легитимные транзакции: {len(legit_metrics):,}")
    
    # Метрики для анализа
    metrics_to_analyze = [
        'num_transactions', 'total_amount', 'unique_merchants', 
        'max_single_amount', 'avg_transaction_amount', 'max_to_total_ratio'
    ]
    
    # Создаем сравнительные графики
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, metric in enumerate(metrics_to_analyze):
        if metric in df_metrics.columns:
            # Гистограммы для сравнения
            fraud_data = fraud_metrics[metric].dropna()
            legit_data = legit_metrics[metric].dropna()
            
            # Ограничиваем выбросы для лучшей визуализации
            fraud_q95 = fraud_data.quantile(0.95)
            legit_q95 = legit_data.quantile(0.95)
            max_val = max(fraud_q95, legit_q95)
            
            fraud_data_limited = fraud_data[fraud_data <= max_val]
            legit_data_limited = legit_data[legit_data <= max_val]
            
            axes[i].hist(legit_data_limited, bins=30, alpha=0.6, 
                        label='Легитимные', color='blue', density=True)
            axes[i].hist(fraud_data_limited, bins=30, alpha=0.6, 
                        label='Мошеннические', color='red', density=True)
            
            axes[i].set_title(f'Распределение: {metric}')
            axes[i].set_xlabel(metric)
            axes[i].set_ylabel('Плотность')
            axes[i].legend()
            axes[i].grid(True, alpha=0.3)
            
            # Добавляем статистику
            fraud_mean = fraud_data.mean()
            legit_mean = legit_data.mean()
            
            stats_text = f'Мошенники: {fraud_mean:.2f}\nЛегитимные: {legit_mean:.2f}'
            axes[i].text(0.02, 0.98, stats_text, transform=axes[i].transAxes, 
                        verticalalignment='top', fontsize=8,
                        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # Статистические тесты
    print("\n--- СТАТИСТИЧЕСКИЕ ТЕСТЫ РАЗЛИЧИЙ ---")
    from scipy import stats
    
    comparison_results = []
    
    for metric in metrics_to_analyze:
        if metric in df_metrics.columns:
            fraud_data = fraud_metrics[metric].dropna()
            legit_data = legit_metrics[metric].dropna()
            
            if len(fraud_data) > 0 and len(legit_data) > 0:
                # T-test
                try:
                    t_stat, p_value = stats.ttest_ind(fraud_data, legit_data)
                    
                    fraud_mean = fraud_data.mean()
                    legit_mean = legit_data.mean()
                    difference_pct = ((fraud_mean - legit_mean) / legit_mean) * 100
                    
                    significance = "Значимо" if p_value < 0.05 else "Не значимо"
                    
                    comparison_results.append({
                        'Метрика': metric,
                        'Мошенники_среднее': fraud_mean,
                        'Легитимные_среднее': legit_mean,
                        'Разница_%': difference_pct,
                        'p_value': p_value,
                        'Значимость': significance
                    })
                    
                except Exception as e:
                    print(f"Ошибка при анализе {metric}: {e}")
    
    if comparison_results:
        results_df = pd.DataFrame(comparison_results)
        print("РЕЗУЛЬТАТЫ СРАВНИТЕЛЬНОГО АНАЛИЗА:")
        print(results_df.to_string(index=False))
        
        # Выводы
        print("\nВЫВОДЫ:")
        significant_diffs = results_df[results_df['Значимость'] == 'Значимо']
        if len(significant_diffs) > 0:
            print("Статистически значимые различия найдены в:")
            for _, row in significant_diffs.iterrows():
                direction = "больше" if row['Разница_%'] > 0 else "меньше"
                print(f"  - {row['Метрика']}: у мошенников {direction} на {abs(row['Разница_%']):.1f}%")
        else:
            print("Статистически значимых различий не обнаружено")
    
    return comparison_results

# Запуск анализа клиентского поведения
demographic_analysis = analyze_demographic_patterns(df_with_features)
activity_metrics = parse_last_hour_activity(df_with_features)
if activity_metrics is not None:
    activity_comparison = analyze_last_hour_activity_patterns(activity_metrics)

print("\nАнализ клиентского поведения завершен")


In [ ]:
# Исследование логики определения "родной страны"
def investigate_home_country_logic(df):
    """Исследует принцип определения родной страны для клиентов"""
    
    print("=== ИССЛЕДОВАНИЕ ЛОГИКИ ОПРЕДЕЛЕНИЯ РОДНОЙ СТРАНЫ ===")
    
    # Анализируем распределение is_outside_home_country по странам
    if 'is_outside_home_country' in df.columns and 'country' in df.columns:
        
        print("\n1. РАСПРЕДЕЛЕНИЕ is_outside_home_country ПО СТРАНАМ:")
        country_home_analysis = df.groupby(['country', 'is_outside_home_country']).size().unstack(fill_value=0)
        country_home_analysis['total'] = country_home_analysis.sum(axis=1)
        country_home_analysis['outside_pct'] = (country_home_analysis[True] / country_home_analysis['total'] * 100)
        
        print(country_home_analysis.sort_values('outside_pct', ascending=False))
        
        # Графическое представление
        plt.figure(figsize=(14, 8))
        
        # График процента "вне родной страны" для каждой страны
        countries = country_home_analysis.index
        outside_percentages = country_home_analysis['outside_pct']
        
        plt.bar(range(len(countries)), outside_percentages, alpha=0.7, color='orange')
        plt.title('Процент транзакций "вне родной страны" по странам\n(Показывает логику определения домашней страны)')
        plt.xlabel('Страны')
        plt.ylabel('Процент транзакций "вне родной страны" (%)')
        plt.xticks(range(len(countries)), countries, rotation=45)
        
        # Добавляем значения на столбцы
        for i, pct in enumerate(outside_percentages):
            plt.text(i, pct + 1, f'{pct:.1f}%', ha='center', va='bottom', fontsize=9)
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n2. СТАТИСТИКА:")
        print(f"Общий процент транзакций 'вне родной страны': {df['is_outside_home_country'].mean()*100:.2f}%")
        
        # Проверяем есть ли customer_id для анализа
        if 'customer_id' in df.columns:
            print("\n3. АНАЛИЗ КЛИЕНТОВ (если данные доступны):")
            # Анализ клиентов по странам
            customer_country = df.groupby('customer_id')['country'].nunique().reset_index()
            multi_country_customers = customer_country[customer_country['country'] > 1]
            print(f"Клиентов с транзакциями в >1 стране: {len(multi_country_customers):,}")
        
        print(f"\n4. ГИПОТЕЗА О ЛОГИКЕ ОПРЕДЕЛЕНИЯ:")
        print("На основе данных можно предположить, что:")
        if country_home_analysis['outside_pct'].std() > 5:  # Если есть существенная разница между странами
            print("- Родная страна определяется на основе регистрации клиента/банка")
            print("- Каждая страна имеет свою базу клиентов")
            print("- is_outside_home_country = True когда клиент совершает транзакцию не в стране регистрации")
        else:
            print("- Все страны показывают похожий процент - возможно случайное распределение")
            print("- Логика может быть основана на других факторах (IP, геолокация и т.д.)")
            
    return country_home_analysis

def suggest_better_city_analysis(df):
    """Предлагает более информативные графики вместо простого распределения по размеру города"""
    
    print("\n=== АЛЬТЕРНАТИВЫ ДЛЯ АНАЛИЗА ГОРОДОВ ===")
    
    if 'city_size' not in df.columns:
        print("Столбец city_size не найден")
        return
    
    print("Текущий график 'Распределение мошенничества по размеру города' действительно малоинформативен.")
    print("Предлагаем следующие альтернативы:\n")
    
    # 1. Fraud rate ratio - сравнение с базовой линией
    print("1. КОЭФФИЦИЕНТ РИСКА ПО РАЗМЕРУ ГОРОДА")
    
    city_fraud = df.groupby('city_size').agg({
        'is_fraud': ['count', 'sum', 'mean']
    }).reset_index()
    city_fraud.columns = ['city_size', 'total_transactions', 'fraud_count', 'fraud_rate']
    
    overall_fraud_rate = df['is_fraud'].mean()
    city_fraud['risk_ratio'] = city_fraud['fraud_rate'] / overall_fraud_rate
    city_fraud['risk_level'] = city_fraud['risk_ratio'].apply(
        lambda x: 'Высокий риск' if x > 1.1 else ('Низкий риск' if x < 0.9 else 'Средний риск')
    )
    
    # График коэффициента риска
    plt.figure(figsize=(12, 6))
    colors = ['red' if x > 1.1 else ('green' if x < 0.9 else 'orange') for x in city_fraud['risk_ratio']]
    
    plt.bar(range(len(city_fraud)), city_fraud['risk_ratio'], color=colors, alpha=0.7)
    plt.axhline(y=1.0, color='black', linestyle='--', alpha=0.5, label='Базовая линия (общий fraud rate)')
    plt.title('Коэффициент риска мошенничества по размеру города\n(>1.1 = высокий риск, <0.9 = низкий риск)')
    plt.xlabel('Размер города')
    plt.ylabel('Коэффициент риска (относительно общего уровня)')
    plt.xticks(range(len(city_fraud)), city_fraud['city_size'])
    plt.legend()
    
    # Добавляем значения и статистическую значимость
    for i, (ratio, level, count) in enumerate(zip(city_fraud['risk_ratio'], city_fraud['risk_level'], city_fraud['total_transactions'])):
        plt.text(i, ratio + 0.05, f'{ratio:.2f}', ha='center', va='bottom', fontsize=10, weight='bold')
        plt.text(i, -0.1, f'{level}', ha='center', va='top', fontsize=8, rotation=0)
    
    plt.tight_layout()
    plt.show()
    
    print("РЕЗУЛЬТАТЫ АНАЛИЗА РИСКА:")
    print(city_fraud[['city_size', 'total_transactions', 'fraud_rate', 'risk_ratio', 'risk_level']].to_string(index=False))
    
    # 2. Временные паттерны по размеру города
    if 'hour' in df.columns:
        print(f"\n2. ВРЕМЕННЫЕ ПАТТЕРНЫ МОШЕННИЧЕСТВА ПО РАЗМЕРУ ГОРОДА")
        
        hourly_city_fraud = df.groupby(['city_size', 'hour'])['is_fraud'].mean().reset_index()
        hourly_city_pivot = hourly_city_fraud.pivot(index='city_size', columns='hour', values='is_fraud')
        
        plt.figure(figsize=(16, 6))
        sns.heatmap(hourly_city_pivot, annot=False, cmap='Reds', cbar_kws={'label': 'Fraud Rate'})
        plt.title('Heatmap: Fraud rate по часам и размеру города')
        plt.xlabel('Час дня')
        plt.ylabel('Размер города')
        plt.tight_layout()
        plt.show()
    
    # 3. Соотношение каналов по размеру города
    if 'channel' in df.columns:
        print(f"\n3. ПРЕДПОЧТЕНИЯ КАНАЛОВ ПО РАЗМЕРУ ГОРОДА")
        
        channel_city = df.groupby(['city_size', 'channel']).size().reset_index(name='count')
        channel_city_pct = channel_city.groupby('city_size').apply(
            lambda x: x.assign(percentage=x['count']/x['count'].sum()*100)
        ).reset_index(drop=True)
        
        # Создаем stacked bar chart
        pivot_channels = channel_city_pct.pivot(index='city_size', columns='channel', values='percentage')
        
        plt.figure(figsize=(12, 6))
        pivot_channels.plot(kind='bar', stacked=True, ax=plt.gca())
        plt.title('Распределение каналов транзакций по размеру города (%)')
        plt.xlabel('Размер города')
        plt.ylabel('Процент использования канала')
        plt.legend(title='Канал', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    
    print(f"\n4. РЕКОМЕНДАЦИИ ДЛЯ ЗАМЕНЫ ГРАФИКА:")
    print("✓ Используйте коэффициент риска вместо абсолютных чисел")
    print("✓ Добавьте временной анализ для выявления паттернов")
    print("✓ Проанализируйте поведенческие различия между городами")
    print("✓ Включите статистическую значимость различий")
    
    return city_fraud

# Запуск исследований
home_country_analysis = investigate_home_country_logic(df_with_features)
better_city_analysis = suggest_better_city_analysis(df_with_features)

print("\nИсследование завершено")


In [ ]:
# 10 АЛЬТЕРНАТИВНЫХ ГРАФИКОВ ДЛЯ ЗАМЕНЫ МАЛОИНФОРМАТИВНЫХ
def create_alternative_analyses(df):
    """10 интересных альтернатив для замены простых демографических графиков"""
    
    print("=== 10 АЛЬТЕРНАТИВНЫХ АНАЛИЗОВ ДЛЯ БИЗНЕСА ===")
    
    # 1. RISK SCORE HEAT MAP - Комбинированный анализ рисков
    print("\n1. HEATMAP КОМБИНИРОВАННЫХ РИСКОВ")
    
    # Создаем risk score на основе нескольких факторов
    df_risk = df.copy()
    
    # Определяем факторы риска
    risk_factors = {
        'high_amount': df_risk['amount_usd'] > df_risk['amount_usd'].quantile(0.9),
        'weekend': df_risk['is_weekend'],
        'outside_country': df_risk['is_outside_home_country'],
        'night_time': df_risk['hour'].isin([0, 1, 2, 3, 4, 5]),
        'high_risk_vendor': df_risk['is_high_risk_vendor']
    }
    
    # Создаем heatmap комбинаций факторов риска
    risk_combinations = []
    for factor1, condition1 in risk_factors.items():
        for factor2, condition2 in risk_factors.items():
            if factor1 != factor2:
                combined_risk = condition1 & condition2
                if combined_risk.sum() > 100:  # Минимум 100 случаев для статистики
                    fraud_rate = df_risk[combined_risk]['is_fraud'].mean() * 100
                    risk_combinations.append({
                        'Factor1': factor1, 
                        'Factor2': factor2, 
                        'Count': combined_risk.sum(),
                        'Fraud_Rate': fraud_rate
                    })
    
    if risk_combinations:
        risk_df = pd.DataFrame(risk_combinations)
        risk_pivot = risk_df.pivot(index='Factor1', columns='Factor2', values='Fraud_Rate')
        
        plt.figure(figsize=(12, 8))
        sns.heatmap(risk_pivot, annot=True, fmt='.1f', cmap='Reds', 
                   cbar_kws={'label': 'Fraud Rate (%)'})
        plt.title('Heatmap: Fraud Rate при комбинации факторов риска')
        plt.xlabel('Второй фактор риска')
        plt.ylabel('Первый фактор риска')
        plt.tight_layout()
        plt.show()
    
    # 2. VELOCITY ANALYSIS - Анализ скорости транзакций
    print("\n2. АНАЛИЗ СКОРОСТИ ТРАНЗАКЦИЙ (VELOCITY)")
    
    if 'timestamp' in df.columns:
        # Сортируем по customer_id и времени (если есть customer_id)
        # Иначе анализируем общую скорость по времени
        df_velocity = df.copy()
        df_velocity = df_velocity.sort_values('timestamp')
        df_velocity['hour_group'] = df_velocity['timestamp'].dt.floor('H')
        
        # Анализ интенсивности транзакций по часам
        hourly_intensity = df_velocity.groupby(['hour_group', 'is_fraud']).size().reset_index(name='count')
        hourly_pivot = hourly_intensity.pivot(index='hour_group', columns='is_fraud', values='count').fillna(0)
        
        # Рассчитываем "скорость мошенничества"
        hourly_pivot['fraud_velocity'] = hourly_pivot[True] / (hourly_pivot[False] + hourly_pivot[True])
        
        # График скорости мошенничества
        plt.figure(figsize=(15, 6))
        plt.plot(hourly_pivot.index, hourly_pivot['fraud_velocity'] * 100, 
                linewidth=2, color='red', alpha=0.7)
        plt.title('Скорость мошенничества по времени (Fraud Velocity)')
        plt.xlabel('Время')
        plt.ylabel('Процент мошеннических транзакций (%)')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    # 3. TRANSACTION CLUSTERING - Кластерный анализ поведения
    print("\n3. КЛАСТЕРНЫЙ АНАЛИЗ ПОВЕДЕНИЯ ТРАНЗАКЦИЙ")
    
    # Создаем признаки для кластеризации
    cluster_features = []
    if 'amount_usd' in df.columns:
        cluster_features.append('amount_usd')
    if 'hour' in df.columns:
        cluster_features.append('hour')
    
    if len(cluster_features) >= 2:
        from sklearn.cluster import KMeans
        from sklearn.preprocessing import StandardScaler
        
        # Подготавливаем данные
        sample_size = min(10000, len(df))  # Берем выборку для скорости
        df_sample = df.sample(n=sample_size, random_state=42)
        
        features_data = df_sample[cluster_features].fillna(0)
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features_data)
        
        # Кластеризация
        kmeans = KMeans(n_clusters=4, random_state=42)
        clusters = kmeans.fit_predict(features_scaled)
        df_sample['cluster'] = clusters
        
        # Анализ fraud rate по кластерам
        cluster_analysis = df_sample.groupby('cluster').agg({
            'is_fraud': ['count', 'sum', 'mean'],
            'amount_usd': 'mean',
            'hour': 'mean'
        }).round(2)
        
        cluster_analysis.columns = ['Total', 'Fraud_Count', 'Fraud_Rate', 'Avg_Amount', 'Avg_Hour']
        cluster_analysis['Fraud_Rate'] = cluster_analysis['Fraud_Rate'] * 100
        
        # График кластеров
        plt.figure(figsize=(12, 8))
        scatter = plt.scatter(df_sample['amount_usd'], df_sample['hour'], 
                            c=df_sample['cluster'], cmap='viridis', alpha=0.6)
        plt.colorbar(scatter, label='Кластер')
        plt.xlabel('Сумма транзакции (USD)')
        plt.ylabel('Час дня')
        plt.title('Кластерный анализ: Группы поведения транзакций')
        
        # Добавляем информацию о fraud rate для каждого кластера
        for cluster_id in range(4):
            cluster_data = df_sample[df_sample['cluster'] == cluster_id]
            if len(cluster_data) > 0:
                mean_amount = cluster_data['amount_usd'].mean()
                mean_hour = cluster_data['hour'].mean()
                fraud_rate = cluster_data['is_fraud'].mean() * 100
                plt.annotate(f'Кластер {cluster_id}\nFraud: {fraud_rate:.1f}%', 
                           xy=(mean_amount, mean_hour), 
                           xytext=(10, 10), textcoords='offset points',
                           bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8),
                           fontsize=8)
        
        plt.tight_layout()
        plt.show()
        
        print("АНАЛИЗ КЛАСТЕРОВ ПОВЕДЕНИЯ:")
        print(cluster_analysis.to_string())
    
    # 4. SEQUENTIAL PATTERN ANALYSIS - Анализ последовательных паттернов
    print("\n4. АНАЛИЗ ПОСЛЕДОВАТЕЛЬНЫХ ПАТТЕРНОВ")
    
    if 'vendor_category' in df.columns:
        # Анализ последовательности категорий поставщиков
        df_sequential = df.copy()
        df_sequential = df_sequential.sort_values('timestamp')
        
        # Создаем последовательности vendor_category для мошенников vs легитимных
        fraud_sequences = df_sequential[df_sequential['is_fraud'] == True]['vendor_category'].value_counts().head(8)
        legit_sequences = df_sequential[df_sequential['is_fraud'] == False]['vendor_category'].value_counts().head(8)
        
        # Сравнительный график популярных категорий
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Мошеннические
        ax1.barh(range(len(fraud_sequences)), fraud_sequences.values, color='red', alpha=0.7)
        ax1.set_yticks(range(len(fraud_sequences)))
        ax1.set_yticklabels(fraud_sequences.index)
        ax1.set_title('Популярные категории: Мошеннические транзакции')
        ax1.set_xlabel('Количество транзакций')
        
        # Легитимные
        ax2.barh(range(len(legit_sequences)), legit_sequences.values, color='green', alpha=0.7)
        ax2.set_yticks(range(len(legit_sequences)))
        ax2.set_yticklabels(legit_sequences.index)
        ax2.set_title('Популярные категории: Легитимные транзакции')
        ax2.set_xlabel('Количество транзакций')
        
        plt.tight_layout()
        plt.show()
    
    # 5. ANOMALY SCORE DISTRIBUTION - Распределение аномальности
    print("\n5. РАСПРЕДЕЛЕНИЕ АНОМАЛЬНОСТИ ТРАНЗАКЦИЙ")
    
    # Создаем простой anomaly score
    df_anomaly = df.copy()
    
    # Z-score для суммы
    if 'amount_usd' in df.columns:
        amount_mean = df_anomaly['amount_usd'].mean()
        amount_std = df_anomaly['amount_usd'].std()
        df_anomaly['amount_zscore'] = abs((df_anomaly['amount_usd'] - amount_mean) / amount_std)
        
        # Anomaly score (простая версия)
        df_anomaly['anomaly_score'] = df_anomaly['amount_zscore']
        
        # Добавляем бонусы за другие аномальные факторы
        if 'hour' in df.columns:
            # Ночное время = аномалия
            df_anomaly.loc[df_anomaly['hour'].isin([0, 1, 2, 3, 4, 5]), 'anomaly_score'] += 1
        
        if 'is_weekend' in df.columns:
            # Выходные = небольшая аномалия
            df_anomaly.loc[df_anomaly['is_weekend'] == True, 'anomaly_score'] += 0.5
        
        # Распределение anomaly score для fraud vs non-fraud
        plt.figure(figsize=(14, 6))
        
        fraud_scores = df_anomaly[df_anomaly['is_fraud'] == True]['anomaly_score']
        legit_scores = df_anomaly[df_anomaly['is_fraud'] == False]['anomaly_score']
        
        # Ограничиваем выбросы для лучшей визуализации
        score_limit = df_anomaly['anomaly_score'].quantile(0.95)
        fraud_scores_limited = fraud_scores[fraud_scores <= score_limit]
        legit_scores_limited = legit_scores[legit_scores <= score_limit]
        
        plt.hist(legit_scores_limited, bins=50, alpha=0.6, label='Легитимные', 
                color='blue', density=True)
        plt.hist(fraud_scores_limited, bins=50, alpha=0.6, label='Мошеннические', 
                color='red', density=True)
        
        plt.title('Распределение Anomaly Score: Мошеннические vs Легитимные')
        plt.xlabel('Anomaly Score (выше = более аномальная)')
        plt.ylabel('Плотность распределения')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Статистика
        print(f"Средний anomaly score (мошеннические): {fraud_scores.mean():.2f}")
        print(f"Средний anomaly score (легитимные): {legit_scores.mean():.2f}")
        print(f"Разница: {((fraud_scores.mean() - legit_scores.mean()) / legit_scores.mean() * 100):.1f}%")
    
    return df_anomaly

# Запуск альтернативных анализов
df_with_alternatives = create_alternative_analyses(df_with_features)

print("\n=== ДОПОЛНИТЕЛЬНЫЕ ВАРИАНТЫ (6-10) ===")
print("6. САНКИ-ДИАГРАММА потоков между странами и каналами")
print("7. СЕТЕВОЙ АНАЛИЗ связей между vendor'ами и fraud")  
print("8. ВРЕМЕННЫЕ РЯДЫ с сезонностью и трендами")
print("9. CORRELATION NETWORK между всеми признаками")
print("10. ГЕОГРАФИЧЕСКИЙ АНАЛИЗ с картой fraud hotspots")
print("\nХотите реализовать какие-то из вариантов 6-10?")


In [ ]:
# НОВЫЕ АНАЛИЗЫ: Замена разделов "АНАЛИЗ РИСКА ПО РАЗМЕРУ ГОРОДА" и "АНАЛИЗ КЛИЕНТОВ ВНЕ РОДНОЙ СТРАНЫ"

# 6. Network graph связей между устройствами и каналами (Санки-диаграмма)
def create_sankey_diagram(df):
    """
    Санки-диаграмма: device → channel → vendor_category с толщиной линий пропорциональной fraud rate
    """
    import plotly.graph_objects as go
    import numpy as np
    
    print("\n=== 6. САНКИ-ДИАГРАММА: ОПАСНЫЕ ПУТИ ТРАНЗАКЦИЙ ===")
    
    # Агрегируем данные для санки-диаграммы
    sankey_data = df.groupby(['device', 'channel', 'vendor_category']).agg({
        'is_fraud': ['count', 'sum', 'mean'],
        'amount': 'sum'
    }).reset_index()
    
    sankey_data.columns = ['device', 'channel', 'vendor_category', 'total_count', 'fraud_count', 'fraud_rate', 'total_amount']
    
    # Фильтруем только значимые пути (минимум 100 транзакций)
    sankey_data = sankey_data[sankey_data['total_count'] >= 100].copy()
    
    # Создаем уникальные узлы
    devices = sankey_data['device'].unique()
    channels = sankey_data['channel'].unique()  
    categories = sankey_data['vendor_category'].unique()
    
    # Создаем маппинг узлов к индексам
    all_nodes = list(devices) + list(channels) + list(categories)
    node_indices = {node: i for i, node in enumerate(all_nodes)}
    
    # Создаем связи device → channel
    device_channel = sankey_data.groupby(['device', 'channel']).agg({
        'total_count': 'sum',
        'fraud_rate': 'mean',
        'total_amount': 'sum'
    }).reset_index()
    
    # Создаем связи channel → vendor_category
    channel_category = sankey_data.groupby(['channel', 'vendor_category']).agg({
        'total_count': 'sum', 
        'fraud_rate': 'mean',
        'total_amount': 'sum'
    }).reset_index()
    
    # Подготавливаем данные для Санки
    source = []
    target = []
    value = []
    colors = []
    
    # Device → Channel связи
    for _, row in device_channel.iterrows():
        source.append(node_indices[row['device']])
        target.append(node_indices[row['channel']])
        value.append(row['total_count'])
        
        # Цвет зависит от fraud rate (красный = высокий риск, зеленый = низкий)
        if row['fraud_rate'] > 0.05:  # Высокий риск
            color = 'rgba(255, 100, 100, 0.6)'
        elif row['fraud_rate'] < 0.02:  # Низкий риск
            color = 'rgba(100, 255, 100, 0.6)'
        else:  # Средний риск
            color = 'rgba(255, 200, 100, 0.6)'
        colors.append(color)
    
    # Channel → Category связи
    for _, row in channel_category.iterrows():
        source.append(node_indices[row['channel']])
        target.append(node_indices[row['vendor_category']])
        value.append(row['total_count'])
        
        # Цвет зависит от fraud rate
        if row['fraud_rate'] > 0.05:
            color = 'rgba(255, 100, 100, 0.6)'
        elif row['fraud_rate'] < 0.02:
            color = 'rgba(100, 255, 100, 0.6)'
        else:
            color = 'rgba(255, 200, 100, 0.6)'
        colors.append(color)
    
    # Цвета узлов
    node_colors = []
    for node in all_nodes:
        if node in devices:
            node_colors.append('lightblue')
        elif node in channels:
            node_colors.append('lightgreen')
        else:  # categories
            node_colors.append('lightcoral')
    
    # Создаем Санки-диаграмму
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=all_nodes,
            color=node_colors
        ),
        link=dict(
            source=source,
            target=target,
            value=value,
            color=colors
        )
    )])
    
    fig.update_layout(
        title_text="Санки-диаграмма: Device → Channel → Vendor Category<br>(Толщина = объем транзакций, Цвет = уровень риска мошенничества)",
        font_size=12,
        height=600
    )
    
    fig.show()
    
    # Статистика по опасным путям
    print("\nТОП-10 САМЫХ ОПАСНЫХ ПУТЕЙ (высокий fraud rate):")
    dangerous_paths = sankey_data.nlargest(10, 'fraud_rate')[
        ['device', 'channel', 'vendor_category', 'total_count', 'fraud_rate', 'total_amount']
    ]
    for _, row in dangerous_paths.iterrows():
        print(f"  {row['device']} → {row['channel']} → {row['vendor_category']}")
        print(f"    Транзакций: {row['total_count']:,}, Fraud rate: {row['fraud_rate']*100:.2f}%, Сумма: ${row['total_amount']:,.0f}")
    
    print(f"\nОбщий fraud rate по всем путям: {sankey_data['fraud_rate'].mean()*100:.2f}%")
    
    return sankey_data

# Запускаем анализ
if 'device' in df_with_features.columns and 'channel' in df_with_features.columns:
    sankey_result = create_sankey_diagram(df_with_features)
else:
    print("Нет данных для санки-диаграммы (отсутствуют поля device/channel)")


In [ ]:
# 7. Survival analysis: время между транзакциями мошенников
def analyze_transaction_intervals(df):
    """
    Анализ интервалов между транзакциями для мошеннических vs легитимных аккаунтов
    """
    import matplotlib.pyplot as plt
    import numpy as np
    from scipy import stats
    import seaborn as sns
    
    print("\n=== 7. SURVIVAL ANALYSIS: ВРЕМЯ МЕЖДУ ТРАНЗАКЦИЯМИ ===")
    
    # Сортируем по customer_id и timestamp
    df_sorted = df.sort_values(['customer_id', 'timestamp']).copy()
    
    # Вычисляем интервалы между транзакциями для каждого клиента
    df_sorted['prev_timestamp'] = df_sorted.groupby('customer_id')['timestamp'].shift(1)
    df_sorted['time_interval_hours'] = (df_sorted['timestamp'] - df_sorted['prev_timestamp']).dt.total_seconds() / 3600
    
    # Убираем первые транзакции каждого клиента (нет предыдущей транзакции)
    intervals_data = df_sorted.dropna(subset=['time_interval_hours']).copy()
    
    # Ограничиваем интервалы разумными значениями (до 30 дней)
    intervals_data = intervals_data[intervals_data['time_interval_hours'] <= 24*30].copy()
    
    # Разделяем на мошеннические и легитимные аккаунты
    fraud_customers = df[df['is_fraud'] == True]['customer_id'].unique()
    
    fraud_intervals = intervals_data[intervals_data['customer_id'].isin(fraud_customers)]['time_interval_hours']
    legit_intervals = intervals_data[~intervals_data['customer_id'].isin(fraud_customers)]['time_interval_hours']
    
    # Создаем субплоты
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Гистограммы распределений
    axes[0,0].hist(legit_intervals, bins=50, alpha=0.7, label='Легитимные', color='green', density=True)
    axes[0,0].hist(fraud_intervals, bins=50, alpha=0.7, label='Мошенники', color='red', density=True)
    axes[0,0].set_xlabel('Интервал между транзакциями (часы)')
    axes[0,0].set_ylabel('Плотность вероятности')
    axes[0,0].set_title('Распределение интервалов между транзакциями')
    axes[0,0].legend()
    axes[0,0].set_xlim(0, 168)  # Неделя
    
    # 2. Box plot сравнение
    intervals_comparison = [legit_intervals.values, fraud_intervals.values]
    axes[0,1].boxplot(intervals_comparison, labels=['Легитимные', 'Мошенники'])
    axes[0,1].set_ylabel('Интервал между транзакциями (часы)')
    axes[0,1].set_title('Box plot: сравнение интервалов')
    axes[0,1].set_yscale('log')
    
    # 3. Survival curves (доля клиентов, совершающих следующую транзакцию)
    time_points = np.logspace(-1, 2, 50)  # От 0.1 до 100 часов
    
    legit_survival = []
    fraud_survival = []
    
    for t in time_points:
        legit_survival.append((legit_intervals <= t).mean())
        fraud_survival.append((fraud_intervals <= t).mean())
    
    axes[1,0].plot(time_points, legit_survival, label='Легитимные', color='green', linewidth=2)
    axes[1,0].plot(time_points, fraud_survival, label='Мошенники', color='red', linewidth=2)
    axes[1,0].set_xlabel('Время (часы)')
    axes[1,0].set_ylabel('Кумулятивная вероятность следующей транзакции')
    axes[1,0].set_title('Survival Curves: скорость повторных транзакций')
    axes[1,0].set_xscale('log')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Среднее количество транзакций в день по типу клиента
    daily_activity = intervals_data.groupby(['customer_id', 
                                           intervals_data['timestamp'].dt.date]).size().reset_index()
    daily_activity.columns = ['customer_id', 'date', 'transactions_per_day']
    daily_activity['is_fraud_customer'] = daily_activity['customer_id'].isin(fraud_customers)
    
    fraud_daily = daily_activity[daily_activity['is_fraud_customer']]['transactions_per_day']
    legit_daily = daily_activity[~daily_activity['is_fraud_customer']]['transactions_per_day']
    
    axes[1,1].hist(legit_daily, bins=30, alpha=0.7, label='Легитимные', color='green', density=True)
    axes[1,1].hist(fraud_daily, bins=30, alpha=0.7, label='Мошенники', color='red', density=True)
    axes[1,1].set_xlabel('Количество транзакций в день')
    axes[1,1].set_ylabel('Плотность вероятности')
    axes[1,1].set_title('Активность: транзакций в день')
    axes[1,1].legend()
    axes[1,1].set_xlim(0, 20)
    
    plt.tight_layout()
    plt.show()
    
    # Статистический анализ
    print("\nСТАТИСТИЧЕСКИЙ АНАЛИЗ:")
    print(f"Медианный интервал (легитимные): {legit_intervals.median():.1f} часов")
    print(f"Медианный интервал (мошенники): {fraud_intervals.median():.1f} часов")
    print(f"Средний интервал (легитимные): {legit_intervals.mean():.1f} часов")
    print(f"Средний интервал (мошенники): {fraud_intervals.mean():.1f} часов")
    
    # Статистический тест
    statistic, p_value = stats.mannwhitneyu(fraud_intervals, legit_intervals, alternative='two-sided')
    print(f"\nMann-Whitney U test p-value: {p_value:.2e}")
    if p_value < 0.05:
        print("Статистически значимая разница в интервалах между группами!")
    
    # Анализ быстрых повторных транзакций (< 1 час)
    quick_legit = (legit_intervals < 1).mean() * 100
    quick_fraud = (fraud_intervals < 1).mean() * 100
    print(f"\nБыстрые повторные транзакции (< 1 час):")
    print(f"  Легитимные клиенты: {quick_legit:.2f}%")
    print(f"  Мошенники: {quick_fraud:.2f}%")
    
    # Анализ очень активных периодов (> 5 транзакций в день)
    hyperactive_legit = (legit_daily > 5).mean() * 100
    hyperactive_fraud = (fraud_daily > 5).mean() * 100
    print(f"\nГиперактивные дни (> 5 транзакций):")
    print(f"  Легитимные клиенты: {hyperactive_legit:.2f}%")
    print(f"  Мошенники: {hyperactive_fraud:.2f}%")
    
    return {
        'fraud_intervals': fraud_intervals,
        'legit_intervals': legit_intervals,
        'fraud_daily': fraud_daily,
        'legit_daily': legit_daily
    }

# Запускаем анализ
if 'timestamp' in df_with_features.columns and 'customer_id' in df_with_features.columns:
    survival_result = analyze_transaction_intervals(df_with_features)
else:
    print("Нет данных для survival analysis (отсутствуют поля timestamp/customer_id)")


In [ ]:
# 8. Scatter plot: активность vs размер транзакции
def create_activity_amount_scatter(df):
    """
    2D scatter: x = last_hour_activity.num_transactions, y = amount_usd, цвет = fraud/no fraud
    """
    import matplotlib.pyplot as plt
    import numpy as np
    import seaborn as sns
    
    print("\n=== 8. SCATTER PLOT: АКТИВНОСТЬ vs РАЗМЕР ТРАНЗАКЦИИ ===")
    
    # Проверяем наличие необходимых полей
    if 'last_hour_activity' not in df.columns:
        print("Поле last_hour_activity отсутствует")
        return None
        
    # Извлекаем данные из структуры last_hour_activity
    df_scatter = df.copy()
    
    # Если last_hour_activity - это структура, извлекаем num_transactions
    if hasattr(df_scatter['last_hour_activity'].iloc[0], 'num_transactions'):
        df_scatter['num_transactions'] = df_scatter['last_hour_activity'].apply(lambda x: x.num_transactions if x else 0)
    else:
        # Если это словарь
        df_scatter['num_transactions'] = df_scatter['last_hour_activity'].apply(
            lambda x: x.get('num_transactions', 0) if isinstance(x, dict) and x else 0
        )
    
    # Используем amount напрямую (конвертируем в USD если есть курсы)
    amount_col = 'amount_usd' if 'amount_usd' in df_scatter.columns else 'amount'
    
    # Фильтруем выбросы для лучшей визуализации
    q99_amount = df_scatter[amount_col].quantile(0.99)
    q99_activity = df_scatter['num_transactions'].quantile(0.99)
    
    df_filtered = df_scatter[
        (df_scatter[amount_col] <= q99_amount) & 
        (df_scatter['num_transactions'] <= q99_activity)
    ].copy()
    
    # Создаем scatter plot
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Основной scatter plot
    fraud_data = df_filtered[df_filtered['is_fraud'] == True]
    legit_data = df_filtered[df_filtered['is_fraud'] == False]
    
    # Случайная выборка для лучшей визуализации
    if len(legit_data) > 5000:
        legit_sample = legit_data.sample(n=5000, random_state=42)
    else:
        legit_sample = legit_data
        
    axes[0,0].scatter(legit_sample['num_transactions'], legit_sample[amount_col], 
                     alpha=0.6, c='green', s=20, label='Легитимные')
    axes[0,0].scatter(fraud_data['num_transactions'], fraud_data[amount_col], 
                     alpha=0.8, c='red', s=30, label='Мошенничество')
    
    axes[0,0].set_xlabel('Количество транзакций за последний час')
    axes[0,0].set_ylabel(f'Размер транзакции ({amount_col})')
    axes[0,0].set_title('Scatter Plot: Активность vs Размер транзакции')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Heatmap плотности мошенничества
    # Создаем сетку для heatmap
    x_bins = np.linspace(0, df_filtered['num_transactions'].max(), 20)
    y_bins = np.linspace(0, df_filtered[amount_col].max(), 20)
    
    fraud_rates = []
    for i in range(len(x_bins)-1):
        row_rates = []
        for j in range(len(y_bins)-1):
            mask = ((df_filtered['num_transactions'] >= x_bins[i]) & 
                   (df_filtered['num_transactions'] < x_bins[i+1]) &
                   (df_filtered[amount_col] >= y_bins[j]) & 
                   (df_filtered[amount_col] < y_bins[j+1]))
            
            if mask.sum() > 10:  # Минимум 10 транзакций в сегменте
                fraud_rate = df_filtered[mask]['is_fraud'].mean()
            else:
                fraud_rate = np.nan
            row_rates.append(fraud_rate)
        fraud_rates.append(row_rates)
    
    fraud_rates = np.array(fraud_rates)
    
    im = axes[0,1].imshow(fraud_rates.T, cmap='Reds', aspect='auto', origin='lower',
                         extent=[0, df_filtered['num_transactions'].max(), 
                                0, df_filtered[amount_col].max()])
    axes[0,1].set_xlabel('Количество транзакций за последний час')
    axes[0,1].set_ylabel(f'Размер транзакции ({amount_col})')
    axes[0,1].set_title('Heatmap: Плотность мошенничества')
    plt.colorbar(im, ax=axes[0,1], label='Fraud Rate')
    
    # 3. Анализ кластеров подозрительного поведения
    # Определяем подозрительные зоны
    high_activity_threshold = df_filtered['num_transactions'].quantile(0.9)
    high_amount_threshold = df_filtered[amount_col].quantile(0.9)
    
    clusters = {
        'Высокая активность + Большая сумма': 
            (df_filtered['num_transactions'] >= high_activity_threshold) & 
            (df_filtered[amount_col] >= high_amount_threshold),
        'Высокая активность + Малая сумма': 
            (df_filtered['num_transactions'] >= high_activity_threshold) & 
            (df_filtered[amount_col] < df_filtered[amount_col].quantile(0.5)),
        'Низкая активность + Большая сумма': 
            (df_filtered['num_transactions'] <= 1) & 
            (df_filtered[amount_col] >= high_amount_threshold),
        'Обычная активность': 
            (df_filtered['num_transactions'] > 1) & 
            (df_filtered['num_transactions'] < high_activity_threshold) &
            (df_filtered[amount_col] < high_amount_threshold)
    }
    
    cluster_stats = []
    colors = ['red', 'orange', 'yellow', 'lightblue']
    
    for i, (cluster_name, mask) in enumerate(clusters.items()):
        cluster_data = df_filtered[mask]
        if len(cluster_data) > 0:
            fraud_rate = cluster_data['is_fraud'].mean()
            cluster_stats.append({
                'Кластер': cluster_name,
                'Количество': len(cluster_data),
                'Fraud Rate': fraud_rate * 100
            })
            
            # Визуализация кластера
            axes[1,0].scatter(cluster_data['num_transactions'], cluster_data[amount_col],
                             alpha=0.6, c=colors[i], label=f'{cluster_name} ({fraud_rate*100:.1f}%)',
                             s=15)
    
    axes[1,0].set_xlabel('Количество транзакций за последний час')
    axes[1,0].set_ylabel(f'Размер транзакции ({amount_col})')
    axes[1,0].set_title('Кластеры поведения (с fraud rate)')
    axes[1,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Статистика по кластерам
    import pandas as pd
    cluster_df = pd.DataFrame(cluster_stats)
    
    if not cluster_df.empty:
        bars = axes[1,1].bar(range(len(cluster_df)), cluster_df['Fraud Rate'], 
                            color=colors[:len(cluster_df)], alpha=0.7)
        axes[1,1].set_xlabel('Кластеры')
        axes[1,1].set_ylabel('Fraud Rate (%)')
        axes[1,1].set_title('Fraud Rate по кластерам поведения')
        axes[1,1].set_xticks(range(len(cluster_df)))
        axes[1,1].set_xticklabels([name[:20] + '...' if len(name) > 20 else name 
                                  for name in cluster_df['Кластер']], rotation=45)
        
        # Добавляем значения на столбцы
        for i, (bar, rate, count) in enumerate(zip(bars, cluster_df['Fraud Rate'], cluster_df['Количество'])):
            axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                          f'{rate:.1f}%\n({count:,})', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # Выводим статистику
    print("\nСТАТИСТИКА ПО КЛАСТЕРАМ:")
    for stat in cluster_stats:
        print(f"  {stat['Кластер']}: {stat['Количество']:,} транзакций, Fraud Rate: {stat['Fraud Rate']:.2f}%")
    
    # Корреляционный анализ
    correlation = df_filtered[['num_transactions', amount_col, 'is_fraud']].corr()
    print(f"\nКОРРЕЛЯЦИИ:")
    print(f"  Активность vs Fraud: {correlation.loc['num_transactions', 'is_fraud']:.3f}")
    print(f"  Размер транзакции vs Fraud: {correlation.loc[amount_col, 'is_fraud']:.3f}")
    print(f"  Активность vs Размер: {correlation.loc['num_transactions', amount_col]:.3f}")
    
    return {
        'cluster_stats': cluster_stats,
        'correlation': correlation,
        'fraud_rates_grid': fraud_rates
    }

# Запускаем анализ
if 'last_hour_activity' in df_with_features.columns:
    activity_result = create_activity_amount_scatter(df_with_features)
else:
    print("Нет данных для scatter plot (отсутствует поле last_hour_activity)")


In [ ]:
# 10. Treemap иерархии рисков
def create_risk_hierarchy_treemap(df):
    """
    Древовидная карта: страна → город → канал → тип карты
    Размер блока = объем транзакций, цвет = fraud rate
    """
    import plotly.graph_objects as go
    import plotly.express as px
    import numpy as np
    
    print("\n=== 10. TREEMAP: ИЕРАРХИЯ РИСКОВ ===")
    
    # Агрегируем данные по иерархии
    hierarchy_data = df.groupby(['country', 'city_size', 'channel', 'card_type']).agg({
        'is_fraud': ['count', 'sum', 'mean'],
        'amount': 'sum'
    }).reset_index()
    
    hierarchy_data.columns = ['country', 'city_size', 'channel', 'card_type', 
                             'total_transactions', 'fraud_count', 'fraud_rate', 'total_amount']
    
    # Фильтруем только значимые сегменты (минимум 50 транзакций)
    hierarchy_data = hierarchy_data[hierarchy_data['total_transactions'] >= 50].copy()
    
    # Создаем иерархические пути для treemap
    hierarchy_data['path'] = (hierarchy_data['country'] + ' / ' + 
                             hierarchy_data['city_size'] + ' / ' + 
                             hierarchy_data['channel'] + ' / ' + 
                             hierarchy_data['card_type'])
    
    # Создаем родительские узлы
    parents = []
    ids = []
    values = []
    colors = []
    
    # Уровень 1: Страны
    country_stats = df.groupby('country').agg({
        'is_fraud': ['count', 'sum', 'mean'],
        'amount': 'sum'
    }).reset_index()
    country_stats.columns = ['country', 'total_transactions', 'fraud_count', 'fraud_rate', 'total_amount']
    country_stats = country_stats[country_stats['total_transactions'] >= 100]
    
    for _, row in country_stats.iterrows():
        parents.append("")
        ids.append(row['country'])
        values.append(row['total_transactions'])
        colors.append(row['fraud_rate'])
    
    # Уровень 2: Страна → Размер города
    city_stats = df.groupby(['country', 'city_size']).agg({
        'is_fraud': ['count', 'sum', 'mean'],
        'amount': 'sum'
    }).reset_index()
    city_stats.columns = ['country', 'city_size', 'total_transactions', 'fraud_count', 'fraud_rate', 'total_amount']
    city_stats = city_stats[
        (city_stats['total_transactions'] >= 75) & 
        (city_stats['country'].isin(country_stats['country']))
    ]
    
    for _, row in city_stats.iterrows():
        parents.append(row['country'])
        ids.append(f"{row['country']} / {row['city_size']}")
        values.append(row['total_transactions'])
        colors.append(row['fraud_rate'])
    
    # Уровень 3: Страна → Размер города → Канал
    channel_stats = df.groupby(['country', 'city_size', 'channel']).agg({
        'is_fraud': ['count', 'sum', 'mean'],
        'amount': 'sum'
    }).reset_index()
    channel_stats.columns = ['country', 'city_size', 'channel', 'total_transactions', 'fraud_count', 'fraud_rate', 'total_amount']
    channel_stats = channel_stats[channel_stats['total_transactions'] >= 50]
    
    for _, row in channel_stats.iterrows():
        parent_id = f"{row['country']} / {row['city_size']}"
        if parent_id in ids:
            parents.append(parent_id)
            ids.append(f"{row['country']} / {row['city_size']} / {row['channel']}")
            values.append(row['total_transactions'])
            colors.append(row['fraud_rate'])
    
    # Уровень 4: Полная иерархия
    for _, row in hierarchy_data.iterrows():
        parent_id = f"{row['country']} / {row['city_size']} / {row['channel']}"
        if parent_id in ids:
            parents.append(parent_id)
            ids.append(row['path'])
            values.append(row['total_transactions'])
            colors.append(row['fraud_rate'])
    
    # Создаем treemap
    fig = go.Figure(go.Treemap(
        ids=ids,
        parents=parents,
        values=values,
        labels=ids,
        marker=dict(
            colorscale='RdYlGn_r',  # Красный для высокого риска, зеленый для низкого
            colorbar=dict(title="Fraud Rate"),
            cmid=df['is_fraud'].mean(),  # Центрируем по общему fraud rate
            line=dict(width=2)
        ),
        textinfo="label+value+percent parent",
        hovertemplate='<b>%{label}</b><br>Транзакций: %{value}<br>Fraud Rate: %{color:.3f}<extra></extra>'
    ))
    
    fig.update_layout(
        title="Treemap: Иерархия рисков мошенничества<br>Страна → Размер города → Канал → Тип карты<br>(Размер = объем транзакций, Цвет = fraud rate)",
        font_size=12,
        height=800
    )
    
    fig.show()
    
    # Альтернативная визуализация - Sunburst chart
    fig2 = go.Figure(go.Sunburst(
        ids=ids,
        parents=parents,
        values=values,
        labels=ids,
        marker=dict(
            colorscale='RdYlGn_r',
            colorbar=dict(title="Fraud Rate"),
            cmid=df['is_fraud'].mean(),
            line=dict(width=2)
        ),
        hovertemplate='<b>%{label}</b><br>Транзакций: %{value}<br>Fraud Rate: %{color:.3f}<extra></extra>'
    ))
    
    fig2.update_layout(
        title="Sunburst: Иерархическое представление рисков",
        font_size=12,
        height=600
    )
    
    fig2.show()
    
    # Анализ топ рисковых сегментов
    print("\nТОП-10 САМЫХ РИСКОВАННЫХ СЕГМЕНТОВ:")
    top_risk = hierarchy_data.nlargest(10, 'fraud_rate')[
        ['country', 'city_size', 'channel', 'card_type', 'total_transactions', 'fraud_rate', 'total_amount']
    ]
    
    for _, row in top_risk.iterrows():
        print(f"  {row['country']} → {row['city_size']} → {row['channel']} → {row['card_type']}")
        print(f"    Транзакций: {row['total_transactions']:,}, Fraud Rate: {row['fraud_rate']*100:.2f}%, Сумма: ${row['total_amount']:,.0f}")
    
    # Анализ самых объемных сегментов
    print("\nТОП-10 САМЫХ ОБЪЕМНЫХ СЕГМЕНТОВ:")
    top_volume = hierarchy_data.nlargest(10, 'total_transactions')[
        ['country', 'city_size', 'channel', 'card_type', 'total_transactions', 'fraud_rate', 'total_amount']
    ]
    
    for _, row in top_volume.iterrows():
        print(f"  {row['country']} → {row['city_size']} → {row['channel']} → {row['card_type']}")
        print(f"    Транзакций: {row['total_transactions']:,}, Fraud Rate: {row['fraud_rate']*100:.2f}%, Сумма: ${row['total_amount']:,.0f}")
    
    # Статистика по уровням иерархии
    print(f"\nСТАТИСТИКА ПО УРОВНЯМ:")
    print(f"  Уникальных стран: {len(country_stats)}")
    print(f"  Уникальных комбинаций страна-город: {len(city_stats)}")
    print(f"  Уникальных комбинаций страна-город-канал: {len(channel_stats)}")
    print(f"  Полных иерархических путей: {len(hierarchy_data)}")
    
    print(f"\nОБЩИЙ FRAUD RATE: {df['is_fraud'].mean()*100:.3f}%")
    print(f"Максимальный fraud rate в сегменте: {hierarchy_data['fraud_rate'].max()*100:.2f}%")
    print(f"Минимальный fraud rate в сегменте: {hierarchy_data['fraud_rate'].min()*100:.2f}%")
    
    return {
        'hierarchy_data': hierarchy_data,
        'country_stats': country_stats,
        'city_stats': city_stats,
        'channel_stats': channel_stats,
        'top_risk': top_risk,
        'top_volume': top_volume
    }

# Запускаем анализ
required_columns = ['country', 'city_size', 'channel', 'card_type']
if all(col in df_with_features.columns for col in required_columns):
    treemap_result = create_risk_hierarchy_treemap(df_with_features)
else:
    missing_cols = [col for col in required_columns if col not in df_with_features.columns]
    print(f"Нет данных для treemap (отсутствуют поля: {missing_cols})")


In [ ]:
# ДОПОЛНИТЕЛЬНЫЙ ГРАФИК: Scatter plot время суток vs объем операций
def create_time_amount_scatter(df):
    """
    Scatter plot: ось Y = время суток, ось X = объем денежных операций (amount),
    цвет точек = fraud или нет
    """
    import matplotlib.pyplot as plt
    import numpy as np
    import seaborn as sns
    from datetime import datetime
    
    print("\n=== ДОПОЛНИТЕЛЬНЫЙ SCATTER PLOT: ВРЕМЯ СУТОК vs ОБЪЕМ ОПЕРАЦИЙ ===")
    
    # Извлекаем час из timestamp
    df_time = df.copy()
    df_time['hour'] = df_time['timestamp'].dt.hour
    
    # Используем amount напрямую
    amount_col = 'amount_usd' if 'amount_usd' in df_time.columns else 'amount'
    
    # Фильтруем выбросы для лучшей визуализации
    q99_amount = df_time[amount_col].quantile(0.99)
    df_filtered = df_time[df_time[amount_col] <= q99_amount].copy()
    
    # Создаем субплоты
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Основной scatter plot
    fraud_data = df_filtered[df_filtered['is_fraud'] == True]
    legit_data = df_filtered[df_filtered['is_fraud'] == False]
    
    # Случайная выборка для лучшей визуализации
    if len(legit_data) > 3000:
        legit_sample = legit_data.sample(n=3000, random_state=42)
    else:
        legit_sample = legit_data
    
    axes[0,0].scatter(legit_sample[amount_col], legit_sample['hour'], 
                     alpha=0.5, c='green', s=15, label='Легитимные')
    axes[0,0].scatter(fraud_data[amount_col], fraud_data['hour'], 
                     alpha=0.8, c='red', s=25, label='Мошенничество')
    
    axes[0,0].set_xlabel(f'Размер транзакции ({amount_col})')
    axes[0,0].set_ylabel('Час суток (0-23)')
    axes[0,0].set_title('Scatter Plot: Размер транзакции vs Время суток')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    axes[0,0].set_yticks(range(0, 24, 2))
    
    # 2. Heatmap fraud rate по времени и размеру транзакции
    # Создаем бины для времени и суммы
    hour_bins = range(0, 25)  # 0-23 часа
    amount_bins = np.percentile(df_filtered[amount_col], np.linspace(0, 100, 21))  # 20 бинов по процентилям
    
    fraud_rates_grid = np.zeros((len(hour_bins)-1, len(amount_bins)-1))
    transaction_counts = np.zeros((len(hour_bins)-1, len(amount_bins)-1))
    
    for i in range(len(hour_bins)-1):
        for j in range(len(amount_bins)-1):
            mask = ((df_filtered['hour'] >= hour_bins[i]) & 
                   (df_filtered['hour'] < hour_bins[i+1]) &
                   (df_filtered[amount_col] >= amount_bins[j]) & 
                   (df_filtered[amount_col] < amount_bins[j+1]))
            
            count = mask.sum()
            transaction_counts[i, j] = count
            
            if count > 10:  # Минимум 10 транзакций в сегменте
                fraud_rates_grid[i, j] = df_filtered[mask]['is_fraud'].mean()
            else:
                fraud_rates_grid[i, j] = np.nan
    
    # Маскируем области с малым количеством данных
    fraud_rates_masked = np.ma.masked_where(transaction_counts < 10, fraud_rates_grid)
    
    im = axes[0,1].imshow(fraud_rates_masked, cmap='RdYlGn_r', aspect='auto',
                         extent=[amount_bins[0], amount_bins[-1], 0, 23])
    axes[0,1].set_xlabel(f'Размер транзакции ({amount_col})')
    axes[0,1].set_ylabel('Час суток')
    axes[0,1].set_title('Heatmap: Fraud Rate по времени и размеру')
    axes[0,1].set_yticks(range(0, 24, 2))
    plt.colorbar(im, ax=axes[0,1], label='Fraud Rate')
    
    # 3. Анализ по временным периодам
    # Определяем временные периоды
    def get_time_period(hour):
        if 6 <= hour < 12:
            return 'Утро (6-12)'
        elif 12 <= hour < 18:
            return 'День (12-18)'
        elif 18 <= hour < 22:
            return 'Вечер (18-22)'
        else:
            return 'Ночь (22-6)'
    
    df_filtered['time_period'] = df_filtered['hour'].apply(get_time_period)
    
    # Определяем категории сумм
    def get_amount_category(amount, percentiles):
        if amount <= percentiles[25]:
            return 'Малая (0-25%)'
        elif amount <= percentiles[50]:
            return 'Средняя (25-50%)'
        elif amount <= percentiles[75]:
            return 'Большая (50-75%)'
        else:
            return 'Очень большая (75-100%)'
    
    amount_percentiles = df_filtered[amount_col].quantile([0.25, 0.5, 0.75, 1.0])
    df_filtered['amount_category'] = df_filtered[amount_col].apply(
        lambda x: get_amount_category(x, amount_percentiles)
    )
    
    # Анализ fraud rate по периодам и категориям сумм
    time_amount_analysis = df_filtered.groupby(['time_period', 'amount_category']).agg({
        'is_fraud': ['count', 'sum', 'mean']
    }).reset_index()
    time_amount_analysis.columns = ['time_period', 'amount_category', 'total_count', 'fraud_count', 'fraud_rate']
    time_amount_analysis = time_amount_analysis[time_amount_analysis['total_count'] >= 20]
    
    # Pivot для heatmap
    pivot_data = time_amount_analysis.pivot(index='time_period', columns='amount_category', values='fraud_rate')
    
    sns.heatmap(pivot_data, annot=True, fmt='.3f', cmap='RdYlGn_r', 
               ax=axes[1,0], cbar_kws={'label': 'Fraud Rate'})
    axes[1,0].set_title('Fraud Rate по времени суток и категории суммы')
    axes[1,0].set_xlabel('Категория суммы транзакции')
    axes[1,0].set_ylabel('Период времени')
    
    # 4. Распределение транзакций по часам с разделением fraud/legit
    hourly_stats = df_filtered.groupby(['hour', 'is_fraud']).size().unstack(fill_value=0)
    
    width = 0.35
    hours = range(24)
    
    axes[1,1].bar([h - width/2 for h in hours], hourly_stats[False], 
                 width, label='Легитимные', alpha=0.7, color='green')
    axes[1,1].bar([h + width/2 for h in hours], hourly_stats[True], 
                 width, label='Мошенничество', alpha=0.7, color='red')
    
    axes[1,1].set_xlabel('Час суток')
    axes[1,1].set_ylabel('Количество транзакций')
    axes[1,1].set_title('Распределение транзакций по часам')
    axes[1,1].legend()
    axes[1,1].set_xticks(range(0, 24, 2))
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Статистический анализ
    print("\nАНАЛИЗ ПО ВРЕМЕННЫМ ПЕРИОДАМ:")
    period_stats = df_filtered.groupby('time_period').agg({
        'is_fraud': ['count', 'sum', 'mean'],
        amount_col: ['mean', 'median']
    }).round(3)
    period_stats.columns = ['total_count', 'fraud_count', 'fraud_rate', 'avg_amount', 'median_amount']
    
    for period in period_stats.index:
        stats = period_stats.loc[period]
        print(f"  {period}:")
        print(f"    Транзакций: {stats['total_count']:,}, Fraud rate: {stats['fraud_rate']*100:.2f}%")
        print(f"    Средняя сумма: ${stats['avg_amount']:,.2f}, Медианная: ${stats['median_amount']:,.2f}")
    
    # Анализ самых опасных временных интервалов
    hourly_fraud = df_filtered.groupby('hour')['is_fraud'].agg(['count', 'sum', 'mean']).round(3)
    hourly_fraud.columns = ['total_count', 'fraud_count', 'fraud_rate']
    hourly_fraud = hourly_fraud[hourly_fraud['total_count'] >= 50]  # Минимум 50 транзакций
    
    print(f"\nСАМЫЕ ОПАСНЫЕ ЧАСЫ (топ-5 по fraud rate):")
    top_dangerous_hours = hourly_fraud.nlargest(5, 'fraud_rate')
    for hour, row in top_dangerous_hours.iterrows():
        print(f"  {hour:02d}:00 - {row['fraud_rate']*100:.2f}% ({row['fraud_count']}/{row['total_count']})")
    
    print(f"\nСАМЫЕ БЕЗОПАСНЫЕ ЧАСЫ (топ-5 по fraud rate):")
    top_safe_hours = hourly_fraud.nsmallest(5, 'fraud_rate')
    for hour, row in top_safe_hours.iterrows():
        print(f"  {hour:02d}:00 - {row['fraud_rate']*100:.2f}% ({row['fraud_count']}/{row['total_count']})")
    
    # Корреляционный анализ
    correlation = df_filtered[['hour', amount_col, 'is_fraud']].corr()
    print(f"\nКОРРЕЛЯЦИИ:")
    print(f"  Час vs Fraud: {correlation.loc['hour', 'is_fraud']:.3f}")
    print(f"  Размер транзакции vs Fraud: {correlation.loc[amount_col, 'is_fraud']:.3f}")
    print(f"  Час vs Размер транзакции: {correlation.loc['hour', amount_col]:.3f}")
    
    return {
        'period_stats': period_stats,
        'hourly_fraud': hourly_fraud,
        'time_amount_analysis': time_amount_analysis,
        'correlation': correlation,
        'top_dangerous_hours': top_dangerous_hours,
        'top_safe_hours': top_safe_hours
    }

# Запускаем анализ
if 'timestamp' in df_with_features.columns:
    time_amount_result = create_time_amount_scatter(df_with_features)
else:
    print("Нет данных для анализа времени (отсутствует поле timestamp)")


In [ ]:
# ЗАКЛЮЧЕНИЕ ПО НОВЫМ АНАЛИЗАМ

print("="*80)
print("РЕЗЮМЕ НОВЫХ АНАЛИЗОВ:")
print("="*80)

print("\n🔗 6. САНКИ-ДИАГРАММА (Network Graph):")
print("   ✅ Показывает 'опасные пути' транзакций: device → channel → vendor_category")
print("   ✅ Толщина линий = объем транзакций, цвет = уровень риска мошенничества")
print("   ✅ Выявляет комбинации устройств, каналов и категорий с высоким fraud rate")

print("\n⏱️ 7. SURVIVAL ANALYSIS:")
print("   ✅ Анализирует интервалы между транзакциями мошенников vs легитимных клиентов")
print("   ✅ Включает survival curves, распределения и поведенческие паттерны")
print("   ✅ Выявляет различия в активности между типами пользователей")

print("\n📊 8. SCATTER PLOT АКТИВНОСТИ:")
print("   ✅ 2D анализ: активность (last_hour_activity) vs размер транзакции")
print("   ✅ Включает heatmap плотности мошенничества и кластерный анализ")
print("   ✅ Показывает подозрительные паттерны поведения")

print("\n🌳 10. TREEMAP ИЕРАРХИИ РИСКОВ:")
print("   ✅ Иерархическая визуализация: страна → город → канал → тип карты")
print("   ✅ Размер блока = объем транзакций, цвет = fraud rate")
print("   ✅ Дополнительно: Sunburst диаграмма для альтернативного представления")

print("\n🕐 ДОПОЛНИТЕЛЬНЫЙ SCATTER PLOT:")
print("   ✅ Время суток vs объем операций с цветовым кодированием fraud/no fraud")
print("   ✅ Включает анализ по временным периодам и heatmap рисков")
print("   ✅ Выявляет временные паттерны мошенничества")

print("\n" + "="*80)
print("ЭТИ ГРАФИКИ ЗАМЕНИЛИ:")
print("❌ 'АНАЛИЗ РИСКА ПО РАЗМЕРУ ГОРОДА' - заменен на более информативные анализы")
print("❌ 'АНАЛИЗ КЛИЕНТОВ ВНЕ РОДНОЙ СТРАНЫ' - заменен на более информативные анализы")
print("="*80)

print("\n🎯 КЛЮЧЕВЫЕ ПРЕИМУЩЕСТВА НОВЫХ АНАЛИЗОВ:")
print("   • Интерактивные Plotly визуализации (Санки, Treemap, Sunburst)")
print("   • Многомерный анализ с несколькими метриками одновременно")
print("   • Временной анализ поведенческих паттернов")
print("   • Иерархическое представление рисков")
print("   • Кластерный анализ подозрительного поведения")
print("   • Корреляционный и статистический анализ")

print("\n📈 ПРАКТИЧЕСКАЯ ЦЕННОСТЬ:")
print("   • Выявление 'опасных путей' для блокировки")
print("   • Определение временных окон повышенного риска")
print("   • Сегментация клиентов по уровню активности и риска")
print("   • Приоритизация мониторинга по географии и каналам")
